In [1]:
"""
l3s4a: per day and all day correlation and other associated analyses of subjective and objective measures (TET and wristband derived respectively)

PART - 1

Combination of l3s2 and l3s3 for subjective and objective measures (TET and wristband per min aggr data derived respectively)

1. Load the script with subjective dimensions like in l3s2 until the big dictionary step.
2. Similarly, load per min agg objective values like in l3s3 until the big dictionary step.
3. Combine the two dictionaries by matching the key values (the days)
4. Correlate all of them with one another and output into corellogram for per day and all day and both with fdr correction - they should all be matched by the same 15 minute bins


PART - 2

Any other useful analyses and visualisations
"""


'\nl3s4a: per day and all day correlation and other associated analyses of subjective and objective measures (TET and wristband derived respectively)\n\nPART - 1\n\nCombination of l3s2 and l3s3 for subjective and objective measures (TET and wristband per min aggr data derived respectively)\n\n1. Load the script with subjective dimensions like in l3s2 until the big dictionary step.\n2. Similarly, load per min agg objective values like in l3s3 until the big dictionary step.\n3. Combine the two dictionaries by matching the key values (the days)\n4. Correlate all of them with one another and output into corellogram for per day and all day and both with fdr correction - they should all be matched by the same 15 minute bins\n\n\nPART - 2\n\nAny other useful analyses and visualisations\n'

In [3]:
#Imports and major functions


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from scipy.stats import shapiro
from scipy.stats import kendalltau
from statsmodels.stats.multitest import multipletests
import seaborn as sns

from l2script_functions import giv_x_y_vals, give_binned_vals, give_binned_vals_category

import warnings
import datetime
from datetime import datetime
import pytz


"""
define binning function for objective measures
"""
def give_binned_vals_obj_meas(df, obj_meas, hour, half_hour, quarter_hour, timezone, category_yn = False):
    
    bin_dict = {}
    bin_dict_scl = {}
    bin_dict_scr = {}

    #only for objective measures, include the extra hour of data (if in cet)
    if category_yn:
        bin_arr = np.arange(0,25,6)
    elif hour: #hour seperation
        bin_arr = np.arange(0,25)
    elif half_hour: #half hour seperation
        bin_arr = np.arange(0,24.5, 0.5)
    elif quarter_hour: #quarter hour seperation
        bin_arr = np.arange(0,24.25, 0.25)
    # Defining the time zones
    utc_zone = pytz.utc
    req_zone = pytz.timezone(timezone)
    
    #if aggr_p_min data, time conversion block (add an extra column to the dataframe with required timezone timestamps)
    def from_isoutc_to_req(iso_timestamp):
            #Parsing the ISO 8601 timestamp into a datetime object
            utc_time = datetime.fromisoformat(iso_timestamp.replace("Z", "+00:00"))
    
            #Converting from utc to required (cet) time
            req_time = utc_time.astimezone(req_zone)
            #print(req_time, type(req_time))
    
            return req_time 

    # Apply the conversion function to the 'utc_timestamps' column and create a new column 'converted_timestamps'
    df['converted_timestamps'] = df['timestamp_iso'].apply(from_isoutc_to_req)

    first_day = df['converted_timestamps'].iloc[0].day 

    x_val = df['converted_timestamps'].apply(
        lambda x: (24 + int(str(x).split()[1].split(':')[0]) + int(str(x).split()[1].split(':')[1])/60) 
        if x.day > first_day 
        else (int(str(x).split()[1].split(':')[0]) + int(str(x).split()[1].split(':')[1])/60)
    ).tolist()

    """
    #Optional, not used so far
    #range of permissible eda range
    min_val = 0.05
    max_val = 60
    #yes but what about scl specifically??? -> haven't found a source talking about scl permissible ranges so for now set this aside
    """
    if obj_meas == 'eda':
        y_val = df['eda_scl_usiemens'].tolist()
    elif obj_meas == 'pulse_rate':
        y_val = df['pulse_rate_bpm'].tolist()
    elif obj_meas == 'prv':
        y_val = df['prv_rmssd_ms'].tolist()
    elif obj_meas == 'resp_rate':
        y_val = df['respiratory_rate_brpm'].tolist()
    elif obj_meas == 'temp':
        y_val = df['temperature_celsius'].tolist()
    elif obj_meas == 'step_count':
        y_val = df['step_counts'].tolist()
    elif obj_meas == 'acc_std':
        y_val = df['accelerometers_std_g'].tolist()
    elif obj_meas == 'activity_counts':
        y_val = df['activity_counts'].tolist()
    elif obj_meas == 'met':
        y_val = df['met'].tolist() 
    elif obj_meas == 'wearing_det':
        y_val = df['wearing_detection_percentage'].tolist()
    ###Optional: include measures after review
    
    for i in range(0, len(bin_arr) - 1):
            #Create the key for the dictionary
            key = str(bin_arr[i]) + '_' + str(bin_arr[i+1])
    
            #Initialize an empty list for this key
            templst = []
            bin_dict[key] = templst
        
            #Iterate over x_val, append to templst if condition is met
            for j in range(0, len(x_val)):
                if x_val[j] >= bin_arr[i] and x_val[j] < bin_arr[i+1] and x_val[j]<bin_arr[-1]: 
                    #Appending y_val[j] directly to the list in the dictionary
                    bin_dict[key].append(y_val[j])
                if x_val[j]>=bin_arr[-1]:
                    #print("not appending value at time: ", x_val[j])
                    continue

    #for conversion of lists to numpy arrays
    for key in bin_dict:
            bin_dict[key] = np.array(bin_dict[key])
            #print(bin_dict)

    bin_dict_mean = {}
    for key in bin_dict:
            if np.all(np.isnan(bin_dict[key])):
                #print('list only has nan values')
                bin_dict_mean[key] = np.nan
            elif ~np.all(np.isnan(bin_dict[key])) and len(bin_dict[key])!=0:
                #print('list is not empty')
                bin_dict_mean[key] = np.nanmean(bin_dict[key]) 
            else:
                print('-5000 has been appended') #this print statement added to debug if there is ever a situation where this would happen (technically it shouldn't)
                bin_dict_mean[key] = -5000

    return  df['converted_timestamps'], x_val, y_val, bin_dict_mean 

ModuleNotFoundError: No module named 'l2script_functions'

In [ ]:
#some more common global variable definitions

folder1 = 'empatica'
folder2 = 'saved_figures'

folder11 = 'aggr_p_min'
folder12 = 'avro_files'
folder13 = 'avro2csv'
folder14 = 'preprocessed_files_debug'
folder141 = 'data_preproc_debug'

ger = True #if country of data collection is Germany, true, else (if UK, false) -> because language of TET questions and other input data config changes by country
timezone = 'Europe/Berlin' # 'utc' #default timezone; enter required timezone if different
mainfolder = input('enter participant folder: ')

In [ ]:
dict_TET_x1, dict_TET_y1,  x_new1, y_new1, x1, y1 = giv_x_y_vals(mainfolder, 'q1', ger) #has duplicates (asd_001)
dict_TET_x2, dict_TET_y2,  x_new2, y_new2, x2, y2 = giv_x_y_vals(mainfolder, 'q2', ger) #has duplicates and days with missing data (asd_001)
dict_TET_x3, dict_TET_y3,  x_new3, y_new3, x3, y3 = giv_x_y_vals(mainfolder, 'q3', ger) #has duplicates (hc_002) #has days with missing data (asd_001)
dict_TET_x4, dict_TET_y4,  x_new4, y_new4, x4, y4 = giv_x_y_vals(mainfolder, 'q4', ger) #has days with missing data (asd_001)
dict_TET_x5, dict_TET_y5,  x_new5, y_new5, x5, y5 = giv_x_y_vals(mainfolder, 'q5', ger) #has duplicates and days with missing data (asd_001)
dict_TET_x6, dict_TET_y6,  x_new6, y_new6, x6, y6 = giv_x_y_vals(mainfolder, 'q6', ger) #has duplicates (hc_002) #has days with missing data (asd_001)
dict_TET_x7, dict_TET_y7,  x_new7, y_new7, x7, y7 = giv_x_y_vals(mainfolder, 'q7', ger)
dict_TET_x8, dict_TET_y8,  x_new8, y_new8, x8, y8 = giv_x_y_vals(mainfolder, 'q8', ger) #has duplicates and days with missing data (asd_001) -> but for the day that it had duplicate data (15_3_24_n7_16_3_24_d) the data was identical so all good
dict_TET_x9, dict_TET_y9,  x_new9, y_new9, x9, y9 = giv_x_y_vals(mainfolder, 'q9', ger) #has duplicates (hc_002) #has days with missing data (asd_001)


In [ ]:
#Now, obtain the binned dictionaries for each dimension

dict_TET_x = [dict_TET_x1, dict_TET_x2, dict_TET_x3, dict_TET_x4, dict_TET_x5, dict_TET_x6, dict_TET_x7, dict_TET_x8, dict_TET_x9]
dict_TET_y = [dict_TET_y1, dict_TET_y2, dict_TET_y3, dict_TET_y4, dict_TET_y5, dict_TET_y6, dict_TET_y7, dict_TET_y8, dict_TET_y9]

dim_q = {}

#pairing up the names of the dimensions for added info
dim_names = ['wakefullness', 'boredom', 'sensory_avoidance', 'social avoidance', 'physical tension', 'scenario_anxiety', 'rumination', 'stress', 'pain'] #can add 'personalised_dimension' as and when it becomes applicable. But most of the times, data is not available
for i in range(9):
    dim_q[f'dim_{i+1}_{dim_names[i]}'] = {}
    for key in dict_TET_x[i]:
        x_val = dict_TET_x[i][key] * 6
        y_val = dict_TET_y[i][key]
        dim_q[f'dim_{i+1}_{dim_names[i]}'][key] = give_binned_vals(x_val, y_val, '15')

In [ ]:
#now for each day, assess each dimension array for normality using shapiro wilk and generate correlograms (pearson and kendall). add an indication for every significant test statistic (p<0.05)

#step 1: for every day, every available dimension assessed for normality and reported in a single dictionary or dataframe

norm_res = {}
for subfolder in os.listdir(mainfolder):
    if subfolder.endswith('d'):
        for i in range(9):
            if subfolder in dim_q[f'dim_{i+1}_{dim_names[i]}'].keys():
                #print('yes ', subfolder,  f'dim_{i+1}_{dim_names[i]}')
                filt_dict = {key: value for key, value in dim_q[f'dim_{i+1}_{dim_names[i]}'][subfolder].items() if value != -5000}
                #print(filt_dict)
                if subfolder not in norm_res:
                        norm_res[subfolder] = {}
                if f'dim_{i+1}_{dim_names[i]}' not in norm_res[subfolder]:
                        norm_res[subfolder][f'dim_{i+1}_{dim_names[i]}'] = {}
                    
                if len(filt_dict) >= 3:
                    stat, p_value = shapiro(list(filt_dict.values()))
                    #print(stat, p_value)
                    
                    
                    norm_res[subfolder][f'dim_{i+1}_{dim_names[i]}']['length of data'] = len(list(filt_dict.values()))
                    norm_res[subfolder][f'dim_{i+1}_{dim_names[i]}']['stat'] = stat
                    norm_res[subfolder][f'dim_{i+1}_{dim_names[i]}']['p_value'] = p_value
                    norm_res[subfolder][f'dim_{i+1}_{dim_names[i]}']['full data'] = dim_q[f'dim_{i+1}_{dim_names[i]}'][subfolder]
                else:
                    norm_res[subfolder][f'dim_{i+1}_{dim_names[i]}']['length of data'] = len(list(filt_dict.values()))
                    norm_res[subfolder][f'dim_{i+1}_{dim_names[i]}']['stat'] = None
                    norm_res[subfolder][f'dim_{i+1}_{dim_names[i]}']['p_value'] = None  # Not enough data for the test
                    norm_res[subfolder][f'dim_{i+1}_{dim_names[i]}']['full data'] = dim_q[f'dim_{i+1}_{dim_names[i]}'][subfolder]

In [ ]:
eda_dict_plot_x = {}
eda_dict_plot_y = {}
eda_dict_bin = {}
pulse_rate_dict_plot_x = {}
pulse_rate_dict_plot_y = {}
pulse_rate_dict_bin = {}
prv_dict_plot_x = {}
prv_dict_plot_y = {}
prv_dict_bin = {}
resp_rate_dict_plot_x = {}
resp_rate_dict_plot_y = {}
resp_rate_dict_bin = {}
temp_dict_plot_x = {}
temp_dict_plot_y = {}
temp_dict_bin = {}
step_dict_plot_x = {}
step_dict_plot_y = {}
step_dict_bin = {}
acc_std_dict_plot_x = {}
acc_std_dict_plot_y = {}
acc_std_dict_bin = {}
activity_dict_plot_x = {}
activity_dict_plot_y = {}
activity_dict_bin = {}
met_dict_plot_x = {}
met_dict_plot_y = {}
met_dict_bin = {}
wearing_det_dict_plot_x = {}
wearing_det_dict_plot_y = {}
wearing_det_dict_bin = {}



#in the lines below, take out "_converted_timestamp," variables after timestamp verification check
eda_converted_timestamp = {}
pulse_rate_converted_timestamp = {}
prv_converted_timestamp = {}
resp_rate_converted_timestamp = {}
temp_converted_timestamp = {}
step_converted_timestamp = {}
acc_converted_timestamp = {}
activity_converted_timestamp = {}
met_converted_timestamp = {}
wearing_det_converted_timestamp = {}

#storing the dates for which the variables are recorded. Required for time-stitching
eda_dates = []
pulse_rate_dates = []
prv_dates = []
resp_rate_dates = []
temp_dates = []
step_dates = []
acc_std_dates = []
activity_dates = []
met_dates = []
wearing_det_dates = []


for subfolder in os.listdir(mainfolder):
    if subfolder.endswith('d') and os.path.exists(os.path.join(mainfolder, subfolder, folder1, folder11)):
        print(subfolder)
        for file in os.listdir(os.path.join(mainfolder, subfolder, folder1, folder11)):
            if file.endswith('eda.csv'):
                eda_df = pd.read_csv(os.path.join(mainfolder, subfolder, folder1, folder11, file))
                eda_converted_timestamp[subfolder], eda_dict_plot_x[subfolder], eda_dict_plot_y[subfolder], eda_dict_bin[subfolder] = give_binned_vals_obj_meas(eda_df, 'eda', False, False, True, timezone)
                eda_dates.append(subfolder)
            
            elif file.endswith('pulse-rate.csv'):
                pulse_rate_df = pd.read_csv(os.path.join(mainfolder, subfolder, folder1, folder11, file))
                pulse_rate_converted_timestamp[subfolder], pulse_rate_dict_plot_x[subfolder], pulse_rate_dict_plot_y[subfolder], pulse_rate_dict_bin[subfolder] = give_binned_vals_obj_meas(pulse_rate_df, 'pulse_rate', False, False, True, timezone)
                pulse_rate_dates.append(subfolder)
            
            elif file.endswith('prv.csv'):
                prv_df = pd.read_csv(os.path.join(mainfolder, subfolder, folder1, folder11, file))
                prv_converted_timestamp[subfolder], prv_dict_plot_x[subfolder], prv_dict_plot_y[subfolder], prv_dict_bin[subfolder] = give_binned_vals_obj_meas(prv_df, 'prv', False, False, True, timezone)
                prv_dates.append(subfolder)
                
            elif file.endswith('respiratory-rate.csv'):
                resp_rate_df = pd.read_csv(os.path.join(mainfolder, subfolder, folder1, folder11, file))
                resp_rate_converted_timestamp[subfolder], resp_rate_dict_plot_x[subfolder], resp_rate_dict_plot_y[subfolder], resp_rate_dict_bin[subfolder] = give_binned_vals_obj_meas(resp_rate_df, 'resp_rate', False, False, True, timezone)
                resp_rate_dates.append(subfolder)
            
            elif file.endswith('temperature.csv'):
                temp_df = pd.read_csv(os.path.join(mainfolder, subfolder, folder1, folder11, file))
                temp_converted_timestamp[subfolder], temp_dict_plot_x[subfolder], temp_dict_plot_y[subfolder], temp_dict_bin[subfolder] = give_binned_vals_obj_meas(temp_df, 'temp', False, False, True, timezone)
                temp_dates.append(subfolder)
            
            elif file.endswith('step-counts.csv'):
                step_df = pd.read_csv(os.path.join(mainfolder, subfolder, folder1, folder11, file))
                step_converted_timestamp[subfolder], step_dict_plot_x[subfolder], step_dict_plot_y[subfolder], step_dict_bin[subfolder] = give_binned_vals_obj_meas(step_df, 'step_count', False, False, True, timezone)
                step_dates.append(subfolder)
                
            elif file.endswith('accelerometers-std.csv'):
                acc_std_df = pd.read_csv(os.path.join(mainfolder, subfolder, folder1, folder11, file))
                acc_converted_timestamp[subfolder], acc_std_dict_plot_x[subfolder], acc_std_dict_plot_y[subfolder], acc_std_dict_bin[subfolder] = give_binned_vals_obj_meas(acc_std_df, 'acc_std', False, False, True, timezone)
                acc_std_dates.append(subfolder)
                
            elif file.endswith('activity-counts.csv'):
                activity_df = pd.read_csv(os.path.join(mainfolder, subfolder, folder1, folder11, file))
                activity_converted_timestamp[subfolder], activity_dict_plot_x[subfolder], activity_dict_plot_y[subfolder], activity_dict_bin[subfolder] = give_binned_vals_obj_meas(activity_df, 'activity_counts', False, False, True, timezone)
                activity_dates.append(subfolder)
            
            elif file.endswith('met.csv'):
                met_df = pd.read_csv(os.path.join(mainfolder, subfolder, folder1, folder11, file))
                met_converted_timestamp[subfolder], met_dict_plot_x[subfolder], met_dict_plot_y[subfolder], met_dict_bin[subfolder] = give_binned_vals_obj_meas(met_df, 'met', False, False, True, timezone)
                met_dates.append(subfolder)
            
            elif file.endswith('wearing-detection.csv'):
                wearing_det_df = pd.read_csv(os.path.join(mainfolder, subfolder, folder1, folder11, file))
                wearing_det_converted_timestamp[subfolder], wearing_det_dict_plot_x[subfolder], wearing_det_dict_plot_y[subfolder], wearing_det_dict_bin[subfolder] = give_binned_vals_obj_meas(wearing_det_df, 'wearing_det', False, False, True, timezone)
                wearing_det_dates.append(subfolder)                
    #(df, obj_meas, hour, half_hour, quarter_hour, timezone):
    
                
            

In [ ]:
#additional function for timestitch
def process_time_values(non_cont_dates, dates_list, dict_plot_x, dict_plot_y):
    """
    Process time values to move data points with x >= 24 to the next day.
    Also creates binned data and calculates bin means for each date.
    
    Parameters:
    non_cont_dates (list): List of dates that are not continuous i.e; dates where the next night is not the very next date but further off. 
    dates_list (list): List of dates for the specific measure
    dict_plot_x (dict): Dictionary with dates as keys and x-values as values
    dict_plot_y (dict): Dictionary with dates as keys and y-values as values
    
    Returns:
    tuple: Modified dict_plot_x, dict_plot_y dictionaries, and bin_dict_mean (nested dictionary)
    """
        
    # Creating copies to avoid modifying the originals directly
    modified_dict_x = {k: v.copy() for k, v in dict_plot_x.items()}
    modified_dict_y = {k: v.copy() for k, v in dict_plot_y.items()}
    
    for i in range(0, len(dates_list)):
        current_date = dates_list[i]
        
        if current_date in non_cont_dates:
            # Discarding values in dict_plot_x[current_date] that are >= 24 and also corresponding values in dict_plot_y[current_date]
            indices_to_remove = [idx for idx, x_val in enumerate(modified_dict_x[current_date]) if x_val >= 24]
            
            # Removing these values from both x and y arrays (in reverse order to avoid index shifts)
            for idx in sorted(indices_to_remove, reverse=True):
                modified_dict_x[current_date].pop(idx)
                modified_dict_y[current_date].pop(idx)

            print(f"Warning: Discarding values >=24 for the this date {current_date} as it is listed in non_cont_dates.")
        else:
            #Checking if this isn't the last date
            if i + 1 < len(dates_list):
                next_date = dates_list[i+1]
                print(current_date, next_date)
                # Finding indices where x values are >= 24
                indices_to_move = [idx for idx, x_val in enumerate(modified_dict_x[current_date]) if x_val >= 24]
                
                if indices_to_move:  # Only process if there are values to move
                    #Values to be moved
                    x_values_to_move = [modified_dict_x[current_date][idx] - 24 for idx in indices_to_move]  # Subtract 24
                    y_values_to_move = [modified_dict_y[current_date][idx] for idx in indices_to_move]
                    
                    #Adding these values to the next day's data
                    modified_dict_x[next_date] = x_values_to_move + modified_dict_x[next_date]
                    modified_dict_y[next_date] = y_values_to_move + modified_dict_y[next_date]
                    
                    #Removing these values from the current day (in reverse order to avoid index shifts)
                    for idx in sorted(indices_to_move, reverse=True):
                        modified_dict_x[current_date].pop(idx)
                        modified_dict_y[current_date].pop(idx)
            else:
                #This is the last date, so we can't move values to the next day
                print(f"Warning: Discarding values >=24 for the last date {current_date} as there's no next day.")
                indices_to_remove = [idx for idx, x_val in enumerate(modified_dict_x[current_date]) if x_val >= 24]
                
                #Remove these values (in reverse order to avoid index shifts)
                for idx in sorted(indices_to_remove, reverse=True):
                    modified_dict_x[current_date].pop(idx)
                    modified_dict_y[current_date].pop(idx)
    
    # Creating binned data for each date
    bin_dict_mean = {}
    
    # Processing each date separately
    for date in dates_list:
        x_val = modified_dict_x[date]
        y_val = modified_dict_y[date]
        
        # Creating bin dictionary for this date
        bin_dict = {}
        
        # Creating bins
        bin_arr = np.arange(0, 24.25, 0.25) #CAUTION: this has to be updated incase a different binning value is chosen
        
        for i in range(0, len(bin_arr) - 1):
            # Creating the key for the dictionary
            key = str(bin_arr[i]) + '_' + str(bin_arr[i+1])
            
            # Initializing an empty list for this key
            templst = []
            bin_dict[key] = templst
            
            # Iterating over x_val, append to templst if condition is met
            for j in range(0, len(x_val)):
                if x_val[j] >= bin_arr[i] and x_val[j] < bin_arr[i+1] and x_val[j] < bin_arr[-1]:
                    # Appending y_val[j] directly to the list in the dictionary
                    bin_dict[key].append(y_val[j])
                if x_val[j] >= bin_arr[-1]:
                    print(f"Date {date}: not appending value at time: {x_val[j]}")
        
        # Converting lists to numpy arrays
        for key in bin_dict:
            bin_dict[key] = np.array(bin_dict[key])
        
        # Calculating means for this date's bins
        date_bin_dict_mean = {}
        for key in bin_dict:
            if len(bin_dict[key]) == 0:
                date_bin_dict_mean[key] = np.nan
            elif np.all(np.isnan(bin_dict[key])):
                # List only has nan values
                date_bin_dict_mean[key] = np.nan
            elif ~np.all(np.isnan(bin_dict[key])) and len(bin_dict[key]) != 0:
                # List is not empty and contains non-nan values
                date_bin_dict_mean[key] = np.nanmean(bin_dict[key]) 
            else:
                print(f'Date {date}: -5000 has been appended')  # Debug statement
                date_bin_dict_mean[key] = -5000
        
        # Adding this date's bin means to the overall dictionary
        bin_dict_mean[date] = date_bin_dict_mean
    
    return modified_dict_x, modified_dict_y, bin_dict_mean

In [ ]:
# Process each measure
non_cont_dates = []  #for asd_001: ["09_3_24_n2_10_3_24_d"] #for hc_002: [] 
eda_dict_plot_x, eda_dict_plot_y, eda_dict_bin = process_time_values(non_cont_dates, eda_dates, eda_dict_plot_x, eda_dict_plot_y)
pulse_rate_dict_plot_x, pulse_rate_dict_plot_y, pulse_rate_dict_bin = process_time_values(non_cont_dates, pulse_rate_dates, pulse_rate_dict_plot_x, pulse_rate_dict_plot_y)
prv_dict_plot_x, prv_dict_plot_y, prv_dict_bin = process_time_values(non_cont_dates, prv_dates, prv_dict_plot_x, prv_dict_plot_y)
resp_rate_dict_plot_x, resp_rate_dict_plot_y, resp_rate_dict_bin = process_time_values(non_cont_dates, resp_rate_dates, resp_rate_dict_plot_x, resp_rate_dict_plot_y)
temp_dict_plot_x, temp_dict_plot_y, temp_dict_bin = process_time_values(non_cont_dates, temp_dates, temp_dict_plot_x, temp_dict_plot_y)
step_dict_plot_x, step_dict_plot_y, step_dict_bin = process_time_values(non_cont_dates, step_dates, step_dict_plot_x, step_dict_plot_y)
acc_std_dict_plot_x, acc_std_dict_plot_y, acc_std_dict_bin = process_time_values(non_cont_dates, acc_std_dates, acc_std_dict_plot_x, acc_std_dict_plot_y)
activity_dict_plot_x, activity_dict_plot_y, activity_dict_bin = process_time_values(non_cont_dates, activity_dates, activity_dict_plot_x, activity_dict_plot_y)
met_dict_plot_x, met_dict_plot_y, met_dict_bin = process_time_values(non_cont_dates, met_dates, met_dict_plot_x, met_dict_plot_y)
wearing_det_dict_plot_x, wearing_det_dict_plot_y, wearing_det_dict_bin = process_time_values(non_cont_dates, wearing_det_dates, wearing_det_dict_plot_x, wearing_det_dict_plot_y)

In [ ]:
#per day correlograms
#need to collect each binned objective measure into separate dictionaries each containing one measure for all days 

list_meas = [eda_dict_bin, pulse_rate_dict_bin, prv_dict_bin, resp_rate_dict_bin, temp_dict_bin, step_dict_bin, acc_std_dict_bin, activity_dict_bin, met_dict_bin, wearing_det_dict_bin]

meas = {}

list_name_meas = ['eda', 'pulse_rate', 'pulse_rate_variability', 'resp_rate', 
                  'temp', 'steps', 'acc_std_dev', 'activity', 
                  'met', 'wearing_det']

list_name_meas1 = ['eda_dict_bin', 'pulse_rate_dict_bin', 'prv_dict_bin', 'resp_rate_dict_bin', 
                  'temp_dict_bin', 'step_dict_bin', 'acc_std_dict_bin', 'activity_dict_bin', 
                  'met_dict_bin', 'wearing_det_dict_bin']

dict_meas = {}

for item_name, item in zip(list_name_meas, list_meas):
    dict_meas[item_name] = item

In [ ]:
#norm_res = {}
for subfolder in os.listdir(mainfolder):
    if subfolder.endswith('d'):
        for i in range(len(list_meas)):
            if subfolder in list_meas[i].keys():
                #print('yes ', subfolder,  list_name_meas[i])
                filt_dict = {key: value for key, value in list_meas[i][subfolder].items() if not np.isnan(value)}
                if subfolder not in norm_res:
                        norm_res[subfolder] = {}
                if list_name_meas[i] not in norm_res[subfolder]:
                        norm_res[subfolder][list_name_meas[i]] = {}
                    
                if len(filt_dict) >= 3:
                    #print(list_name_meas[i])
                    stat, p_value = shapiro(list(filt_dict.values())) #sometimes issues warning saying range 0 and so reslt can be inaccurate. But couldn't figre out which measre as this warning does not appear consistently
                    #print(stat, p_value)                  

                    norm_res[subfolder][list_name_meas[i]]['length of data (without nans)'] = len(list(filt_dict.values()))
                    norm_res[subfolder][list_name_meas[i]]['stat'] = stat
                    norm_res[subfolder][list_name_meas[i]]['p_value'] = p_value
                    norm_res[subfolder][list_name_meas[i]]['full data'] = list_meas[i][subfolder]
                else:
                    norm_res[subfolder][list_name_meas[i]]['length of data (without nans)'] = len(list(filt_dict.values()))
                    norm_res[subfolder][list_name_meas[i]]['stat'] = None
                    norm_res[subfolder][list_name_meas[i]]['p_value'] = None  # Not enough data for the test
                    norm_res[subfolder][list_name_meas[i]]['full data'] = list_meas[i][subfolder]

In [ ]:
#Optional: in case plots don't load right away, this is a good debug strategy
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Additional imports if not already executed
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
"""
Plot all days data on the same graph sequentially
"""

In [ ]:
#conda install anaconda::mpld3 -> didn't work, gave error. discard
#!pip install mpld3 -> no need; did not work
#run below if plotly not available
!pip install plotly matplotlib

In [ ]:
#Optional imports if not imported already
import matplotlib
print(matplotlib.__version__)

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from datetime import datetime

In [ ]:
#interactive plotting and incorporating dates and days of the week

# Function to get date and weekday label
def giv_date_and_day(reqYear, reqMonth, reqDay):
    date_obj = datetime(reqYear, reqMonth, reqDay)
    return date_obj.strftime("%Y-%m-%d\n%a")  # Multi-line label: Date + weekday abbrev

# Prepare figure for 3 stacked subplots with shared x-axis
fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True,
    subplot_titles=(
        "Subjective Dimensions (Day-stacked)",
        "Objective Measures (Day-stacked), set 1",
        "Objective Measures (Day-stacked), set 2"
    )
)

# Keys and colors
dim_keys = [f"dim_{i}_{name}" for i, name in enumerate([
    "wakefullness", "boredom", "sensory_avoidance", "social avoidance",
    "physical tension", "scenario_anxiety", "rumination", "stress", "pain"
], start=1)]
core_physio_keys = ['eda', 'pulse_rate', 'resp_rate', 'pulse_rate_variability', 'temp']
movement_keys = ['steps', 'acc_std_dev', 'activity', 'met', 'wearing_det']
colors = ['blue', 'orange', 'green', 'grey', 'purple', 'brown', 'pink', 'red', 'olive']

x_offset = 0
day_boundaries = []
day_labels = []
special_bins = [("12.0_12.25", "12:00"), ("18.0_18.25", "18:00")]

partDates = list(norm_res.keys())

for day_idx, day_key in enumerate(partDates):
    day_data = norm_res[day_key]
    first_dim = list(day_data.keys())[0]
    bins = list(day_data[first_dim]['full data'].keys())
    x_vals = np.arange(len(bins)) + x_offset

    # Extract date info from day key to generate tick label
    reqYear = 2000 + int(day_key.split('_')[-2])
    reqMonth = int(day_key.split('_')[-3])
    reqDay = int(day_key.split('_')[-4])
    day_label = giv_date_and_day(reqYear, reqMonth, reqDay)

    day_boundaries.append(x_vals[-1])  # position of day boundary for ticks
    print('day_boundaries = ', day_boundaries)
    day_labels.append(day_label)       # label string for that position

    # Plot dim_keys
    for i, dim in enumerate(dim_keys):
        if dim in day_data:
            y_raw = list(day_data[dim]['full data'].values())
            y_vals = [np.nan if v is None or (isinstance(v, float) and np.isnan(v)) or v == -5000 else v for v in y_raw]
            fig.add_trace(go.Scatter(
                x=x_vals, y=y_vals, mode='lines',
                name=dim, legendgroup=dim, showlegend=(day_idx == 6),
                line=dict(color=colors[i % len(colors)], width=1.5)), row=1, col=1)

    # Plot core_physio_keys
    for i, ph in enumerate(core_physio_keys):
        if ph in day_data and "full data" in day_data[ph]:
            y_raw = list(day_data[ph]['full data'].values())
            y_vals = [np.nan if v is None or (isinstance(v, float) and np.isnan(v)) or v == -5000 else v for v in y_raw]
            fig.add_trace(go.Scatter(
                x=x_vals, y=y_vals, mode='lines',
                name=ph, legendgroup=ph, showlegend=(day_idx == 6),
                line=dict(color=colors[i % len(colors)], width=1.5)), row=2, col=1)
    
    # Plot movement_keys
    for i, ph in enumerate(movement_keys):
        if ph in day_data and "full data" in day_data[ph]:
            y_raw = list(day_data[ph]['full data'].values())
            y_vals = [np.nan if v is None or (isinstance(v, float) and np.isnan(v)) or v == -5000 else v for v in y_raw]
            fig.add_trace(go.Scatter(
                x=x_vals, y=y_vals, mode='lines',
                name=ph, legendgroup=ph, showlegend=(day_idx == 6),
                line=dict(color=colors[i % len(colors)], width=1.5)), row=3, col=1)
    
    # Vertical dashed lines and annotations for special bins
    for special_bin, label in special_bins:
        if special_bin in bins:
            idx = bins.index(special_bin)
            xpos = x_vals[idx]
            fig.add_shape(type="line", x0=xpos, x1=xpos, y0=0, y1=1, yref="paper",
                          line=dict(color="grey", width=1, dash="dot"))
            fig.add_annotation(x=xpos, y=1.05, yref="paper", text=label,
                               showarrow=False, textangle=90, yanchor='bottom')

    x_offset += len(bins)
    print('x_offset = ', x_offset)

# Dashed black vertical day boundary lines
for boundary in day_boundaries:
    fig.add_shape(type="line", x0=boundary, x1=boundary, y0=0, y1=1, yref="paper",
                  line=dict(color="black", width=1, dash="dash"))

participant_title_list = mainfolder.split("\\")[-1].split('_')[0:3]
participant_title = '_'.join(participant_title_list)

fig.update_layout(
    height=900,
    width=1200,
    title_text="Participant " +  participant_title,
    legend=dict(yanchor="middle", y=0.9, xanchor='left', x=1.05),
   
    xaxis3=dict(
        tickmode="array",
        tickvals=day_boundaries,
        ticktext=day_labels,
        tickangle=-45,
        tickfont=dict(size=10),
        showgrid=True,
        zeroline=False
    )
)

# Axis labels
fig.update_xaxes(title_text="Time (concatenated bins across days)", row=3, col=1)
fig.update_yaxes(title_text="Value")

# Save interactive html
#modify focus_folder as required
focus_folder = r'dataclouds_CAM_LMU\Stream_HC_002\all_days'

fig.write_html(os.path.join(focus_folder, "updated_all_days_stacked_fifteen_min_timestitch_interactive_plot_w_days_.html"))

# Show plot
fig.show()


In [ ]:
"""
PER DAY CORELLOGRAM - CAN MODIFY THE INCLUDED SUBJECTIVE/OBJECTIVE PARAMETERS FOR FOCUSED VISUALISATION

SAVES FIGURES, INTEGRATES ALL INFO IN EACH CELL

ALSO TRIES DATACLOUDS

Choose from the measures listed below if you only want to visualise and check correlations of a subset of measures. List the excluded ones in "measures_to_pop" and uncomment the line

#['dim_1_wakefullness', 'dim_2_boredom', 'dim_3_sensory_avoidance', 'dim_4_social avoidance', 'dim_5_physical tension', 'dim_6_scenario_anxiety', 'dim_7_rumination', 'dim_8_stress', 'dim_9_pain', 'eda', 'pulse_rate', 'pulse_rate_variability', 'resp_rate', 'temp', 'steps', 'acc_std_dev', 'activity', 'met', 'wearing_det']
measures_to_pop = ['dim_2_boredom', 'dim_3_sensory_avoidance', 'dim_4_social avoidance', 'dim_6_scenario_anxiety', 'temp', 'steps', 'activity', 'met', 'wearing_det'] 

#specify the folder for savefig line in daily datacloud function

"""


def giv_date_and_day(reqYear, reqMonth, reqDay):
    date_obj = datetime(reqYear, reqMonth, reqDay)
    return date_obj.strftime("%Y-%m-%d\n%a")  
    
def create_daily_datacloud_matrix(df, subfolder, mainfolder, folder2, kendall_mat, kendall_pval_mat, kendall_pval_mat_fdr, kendall_len_mat, min_points=1):
    """
    Create a matrix of scatter plots (dataclouds) for daily correlation analysis
    Now uses FDR-corrected p-values for significance
    """
    #special_var: dictionary of variables that have a certain upper limit (inferred from their daily traces and plots). If they are included in the dictionary below, the plots will have these x and y upper limits. If not the upper limits will go upto 4 (the range of TET magnitude values)
    special_var = {'eda': 45, 'pulse_rate':250, 'pulse_rate_variability':300, 'resp_rate':30, 'acc_std_dev':1} #this can also vary from participant to participant. Verify and update values
    # Filter out columns with all NaN or insufficient data
    valid_columns = []
    for col in df.columns:
        if df[col].dropna().shape[0] >= 3:  # Need at least 3 points
            valid_columns.append(col)
    
    if len(valid_columns) < 2:
        print(f"Insufficient data for dataclouds on {subfolder}")
        return
    
    n_vars = len(valid_columns)
    
    # Create figure with subplots
    fig, axes = plt.subplots(n_vars, n_vars, figsize=(4*n_vars, 4*n_vars))
    suptitle = f'Daily Datacloud for {subfolder}'
    fig.suptitle(suptitle, fontsize=16, y=0.98)
    
    # If only one variable, make axes 2D for consistent indexing
    if n_vars == 1:
        axes = np.array([[axes]])
    elif n_vars == 2:
        axes = axes.reshape(2, 2)
    
    for i, dim1 in enumerate(valid_columns):
        for j, dim2 in enumerate(valid_columns):
            ax = axes[i, j]
            
            # Get filtered data for this pair
            filt_data = df[[dim1, dim2]].dropna()
            
            if len(filt_data) >= 3:
                if i == j:  # Diagonal - leave blank
                    ax.text(0.5, 0.5, f'Diagonal\n', 
                           transform=ax.transAxes, ha='center', va='center',
                           fontsize=10, color='red')
                    ax.set_xlabel(dim1, fontsize=9)
                    ax.set_ylabel(dim2, fontsize=9)
                elif i>j:  # Off-diagonal - show scatter plot
                    x_data = filt_data[dim1]
                    y_data = filt_data[dim2]
                    # Create scatter plot
                    ax.scatter(x_data, y_data, alpha=0.6, s=20, color='steelblue')
                    if dim1 in special_var.keys():
                        for key, val in special_var.items():
                            if dim1 == key:
                                ax.set_xlim(0, val)
                    else:
                        ax.set_xlim(0, 4)
                    if dim2 in special_var.keys():
                        for key, val in special_var.items():
                            if dim2 == key:
                                ax.set_ylim(0, val)
                    else:
                        ax.set_ylim(0, 4)
                    # Get correlation statistics from existing matrices
                    if len(filt_data) >= min_points and not np.isnan(kendall_mat.loc[dim1, dim2]):
                        corr_ken = kendall_mat.loc[dim1, dim2]
                        p_ken_raw = kendall_pval_mat.loc[dim1, dim2]
                        p_ken_fdr = kendall_pval_mat_fdr.loc[dim1, dim2]
                        n_points = int(kendall_len_mat.loc[dim1, dim2])
                        
                        # Add trend line if FDR-corrected correlation is significant and reasonably strong
                        if not np.isnan(p_ken_fdr) and p_ken_fdr < 0.05 and abs(corr_ken) > 0.2:
                            # Fit linear trend line
                            z = np.polyfit(x_data, y_data, 1)
                            p = np.poly1d(z)
                            ax.plot(x_data.sort_values(), p(x_data.sort_values()), 
                                   "r--", alpha=0.8, linewidth=1.5)
                        
                        # Add correlation info with both raw and FDR p-values
                        fdr_sig = "*" if (not np.isnan(p_ken_fdr) and p_ken_fdr < 0.05) else ""
                        raw_sig = "*" if p_ken_raw < 0.05 else ""
                        
                        annotation_text = f'tau={corr_ken:.3f}{fdr_sig}\n'
                        annotation_text += f'p_raw={p_ken_raw:.3f}{raw_sig}\n'
                        if not np.isnan(p_ken_fdr):
                            annotation_text += f'p_fdr={p_ken_fdr:.3f}{fdr_sig}\n'
                        else:
                            annotation_text += f'p_fdr=n/a\n'
                        annotation_text += f'n={n_points}'
                        
                        ax.text(0.05, 0.95, annotation_text,
                               transform=ax.transAxes, fontsize=7,
                               verticalalignment='top',
                               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
                    
                    ax.set_xlabel(dim1, fontsize=9)
                    ax.set_ylabel(dim2, fontsize=9)
                else: 
                    # Upper triangle; need not be repeated
                    ax.text(0.5, 0.5, f'Upper triangle\n', 
                           transform=ax.transAxes, ha='center', va='center',
                           fontsize=10, color='red')
                    ax.set_xlabel(dim1, fontsize=9)
                    ax.set_ylabel(dim2, fontsize=9)
                    
            else:
                # Not enough data
                ax.text(0.5, 0.5, f'Insufficient data\n(n={len(filt_data)})', 
                       transform=ax.transAxes, ha='center', va='center',
                       fontsize=10, color='red')
                ax.set_xlabel(dim1, fontsize=9)
                ax.set_ylabel(dim2, fontsize=9)
                if dim1 == special_var:
                    ax.set_xlim(0, 45)
                else:
                    ax.set_xlim(0, 4)
                if dim2 == special_var:
                    ax.set_ylim(0, 45)
                else:
                    ax.set_ylim(0, 4)
            
            # Clean up axes
            ax.tick_params(labelsize=8)
            ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    #specify the folder to save the figures. If folder does not exist, create it. 
    focused_folder = folder2 
    plt.savefig(os.path.join(focused_folder, f"v2_focused_daily_datacloud_fdr_{subfolder}_timestitch.png"),bbox_inches='tight', dpi=300)
    plt.show()


def create_daily_condensed_datacloud_matrix(df, subfolder, mainfolder, folder2, kendall_mat, kendall_pval_mat, kendall_pval_mat_fdr, kendall_len_mat, min_points=1):
    """
    Optional added function to plot the same plots as above but in order of their correlation strength from highest to lowest. Can also add (copy-paste) the x and y limit logic from the previous function to this one if using
    Create a condensed matrix showing only significant daily correlations (lower triangle)
    Now uses FDR-corrected p-values for significance
    """
    # Filter out columns with all NaN or insufficient data
    valid_columns = []
    for col in df.columns:
        if df[col].dropna().shape[0] >= 3:
            valid_columns.append(col)
    
    if len(valid_columns) < 2:
        print(f"Need at least 2 variables for correlation dataclouds on {subfolder}")
        return
    
    # Calculate number of unique pairs
    n_vars = len(valid_columns)
    n_unique_pairs = n_vars * (n_vars - 1) // 2
    
    if n_unique_pairs == 0:
        print(f"Need at least 2 variables for correlation dataclouds on {subfolder}")
        return
    
    # Determine subplot layout
    cols = min(4, n_unique_pairs)  # Max 4 columns
    rows = (n_unique_pairs + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 4*rows))
    suptitle = f'Daily Significant Correlations (FDR-corrected) - Dataclouds for {subfolder}'
    fig.suptitle(suptitle, fontsize=14)
    
    # Make axes iterable
    if n_unique_pairs == 1:
        axes = [axes]
    else:
        axes = axes.flatten() if hasattr(axes, 'flatten') else axes
    
    plot_idx = 0
    significant_pairs = []
    
    # Collect all pairs first (lower triangle)
    for i, dim1 in enumerate(valid_columns):
        for j, dim2 in enumerate(valid_columns):
            if i <= j:  # Only lower triangle
                continue
                
            # Get filtered data
            filt_data = df[[dim1, dim2]].dropna()
            
            if len(filt_data) >= min_points and not np.isnan(kendall_mat.loc[dim1, dim2]):
                corr_ken = kendall_mat.loc[dim1, dim2]
                p_ken_raw = kendall_pval_mat.loc[dim1, dim2]
                p_ken_fdr = kendall_pval_mat_fdr.loc[dim1, dim2]
                n_points = int(kendall_len_mat.loc[dim1, dim2])
                significant_pairs.append((dim1, dim2, filt_data, corr_ken, p_ken_raw, p_ken_fdr, n_points))
    
    # Sort by absolute correlation strength
    significant_pairs.sort(key=lambda x: abs(x[3]), reverse=True)
    
    # Plot significant pairs
    for dim1, dim2, filt_data, corr_ken, p_ken_raw, p_ken_fdr, n_points in significant_pairs[:len(axes)]:
        ax = axes[plot_idx]
        
        x_data = filt_data[dim1]
        y_data = filt_data[dim2]
        
        # Create scatter plot
        ax.scatter(x_data, y_data, alpha=0.6, s=30, color='steelblue')
        
        # Add trend line for FDR-significant correlations
        if not np.isnan(p_ken_fdr) and p_ken_fdr < 0.05:
            z = np.polyfit(x_data, y_data, 1)
            p = np.poly1d(z)
            x_trend = np.linspace(x_data.min(), x_data.max(), 100)
            ax.plot(x_trend, p(x_trend), "r-", alpha=0.8, linewidth=2)
        
        # Add statistics with FDR information
        fdr_significance = "*" if (not np.isnan(p_ken_fdr) and p_ken_fdr < 0.05) else ""
        raw_significance = "*" if p_ken_raw < 0.05 else ""
        
        title_text = f'{dim1} vs {dim2}\ntau={corr_ken:.3f}{fdr_significance}'
        subtitle_text = f'p_raw={p_ken_raw:.3f}{raw_significance}'
        if not np.isnan(p_ken_fdr):
            subtitle_text += f', p_fdr={p_ken_fdr:.3f}{fdr_significance}'
        subtitle_text += f', n={n_points}'
        
        ax.set_title(title_text, fontsize=10, fontweight='bold')
        ax.text(0.5, -0.15, subtitle_text, transform=ax.transAxes, 
                ha='center', fontsize=8, style='italic')
        
        ax.set_xlabel(dim1, fontsize=9)
        ax.set_ylabel(dim2, fontsize=9)
        ax.grid(True, alpha=0.3)
        
        plot_idx += 1
    
    # Hide unused subplots
    for idx in range(plot_idx, len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    plt.savefig(os.path.join(mainfolder, subfolder, folder2, f"filtered_daily_significant_dataclouds_fdr_{subfolder}_timestitch.png"), bbox_inches='tight', dpi=300)
    plt.show()
    return
    
   


# Main per-day analysis loop with integrated dataclouds
kendall_mat_dict = {}
kendall_pval_mat_dict = {}
kendall_pval_mat_fdr_dict = {}
kendall_len_mat_dict = {}
shapiro_pval_mat_dict = {}

for subfolder in norm_res.keys():
    if subfolder == '25_3_24_n17_26_3_24_d': #this if statement is for a specific participant (ASD_001) because there was no recorded subjective TET data recorded for this day. Modufy this if necessary for other participants if applicable or keep it as is, code executes regardless
        continue
    else:
        dim_dict = {}
        for dim_qx in norm_res[subfolder].keys():
            if dim_qx in measures_to_pop:
                continue
                
            dim_dict[dim_qx] = norm_res[subfolder][dim_qx]['full data']
        df = pd.DataFrame(dim_dict) 
        #now need a corellogram such that for values =-5000 are turned to NaN and dropped only for the two columns being correlated. Kendall correlation using scipy needs to exceute
        df.replace(-5000, np.nan, inplace=True)
        #shapiro_mat = pd.DataFrame(np.zeros((df.shape[1], df.shape[1])), columns=df.columns, index=df.columns) - optional, not necessary
        shapiro_pval_mat = pd.DataFrame(np.zeros((df.shape[1], df.shape[1])), columns=df.columns, index=df.columns)
        kendall_mat = pd.DataFrame(np.zeros((df.shape[1], df.shape[1])), columns=df.columns, index=df.columns)
        kendall_pval_mat = pd.DataFrame(np.zeros((df.shape[1], df.shape[1])), columns=df.columns, index=df.columns)
        kendall_len_mat = pd.DataFrame(np.zeros((df.shape[1], df.shape[1])), columns=df.columns, index=df.columns)
    
        #for fdr correction
        kendall_pval_mat_fdr = pd.DataFrame(np.zeros((df.shape[1], df.shape[1])), columns=df.columns, index=df.columns)
        
        for dim1 in df.columns:
            for dim2 in df.columns:
                filt_data = df[[dim1, dim2]].dropna()
    
                if len(filt_data)>=3:
                    stat1, pval1 = shapiro(filt_data[dim1])
                    stat2, pval2 = shapiro(filt_data[dim2])
                    corr_ken, p_ken = kendalltau(filt_data[dim1], filt_data[dim2])
                    if pval1 > 0.05 and pval2 > 0.05:
                        shapiro_pval_mat.loc[dim1, dim2] = 1
                    else:
                        shapiro_pval_mat.loc[dim1, dim2] = 0
                    kendall_mat.loc[dim1, dim2] = corr_ken
                    kendall_pval_mat.loc[dim1,dim2] = p_ken
                    kendall_len_mat.loc[dim1,dim2] = len(filt_data)
                else:
                    #not enough data to test normality or calculate kendall corr
                    shapiro_pval_mat.loc[dim1, dim2] = np.nan
                    kendall_mat.loc[dim1, dim2] = np.nan
                    kendall_pval_mat.loc[dim1,dim2] = np.nan
                    kendall_len_mat.loc[dim1,dim2] = len(filt_data)
    
        #Applying fdr correction
        temp_store = []
        pPreFdr = []
        pos = []
        for meas_1 in df.columns:
            temp_store.append(meas_1)
            for meas_2 in df.columns:
                if meas_1 == meas_2:
                    continue
                elif meas_2 in temp_store:
                    continue
                else:
                    if not np.isnan(kendall_mat.loc[meas_1, meas_2]):
                        pPreFdr.append(kendall_pval_mat.loc[meas_1, meas_2])
                        pos.append((meas_1, meas_2))
        
        pPostFdr = multipletests(pPreFdr, method='fdr_bh')[1]
        
        for (meas_1, meas_2), p_adj in zip(pos, pPostFdr):
            kendall_pval_mat_fdr.loc[meas_1, meas_2] = p_adj
            kendall_pval_mat_fdr.loc[meas_2, meas_1] = p_adj
        
        #fill diagonal and any missing values with NaN 
        for meas in df.columns:
            kendall_pval_mat_fdr.loc[meas, meas] = np.nan
    
        #storing values per day
        kendall_mat_dict[subfolder] = kendall_mat.copy()
        kendall_pval_mat_dict[subfolder] = kendall_pval_mat.copy()
        kendall_pval_mat_fdr_dict[subfolder] = kendall_pval_mat_fdr.copy()
        kendall_len_mat_dict[subfolder] = kendall_len_mat.copy()
        shapiro_pval_mat_dict[subfolder] = shapiro_pval_mat.copy()
    
        #creating a custom annotation matrix combining tau, p-value, and data length
        annotations = kendall_mat.copy()
        
        for dim1 in df.columns:
            for dim2 in df.columns:
                if not np.isnan(kendall_mat.loc[dim1, dim2]):
                    annotations.loc[dim1, dim2] = f"tau={kendall_mat.loc[dim1, dim2]:.2f}\n"
                    if kendall_pval_mat.loc[dim1, dim2] < 0.05:
                        annotations.loc[dim1, dim2] += f"p={kendall_pval_mat.loc[dim1, dim2]:.2f}*\n"
                    else:
                        annotations.loc[dim1, dim2] += f"p={kendall_pval_mat.loc[dim1, dim2]:.2f}\n"
                    annotations.loc[dim1, dim2] += f"dl={int(kendall_len_mat.loc[dim1, dim2])}\n"
                    annotations.loc[dim1, dim2] += f"norm={int(shapiro_pval_mat.loc[dim1, dim2])}"
                else:
                    annotations.loc[dim1, dim2] = ""
    
        # Plot the combined heatmap with custom annotations (tau, p-value, and data length)
        plt.figure(figsize=(32, 32))
        sns.heatmap(kendall_mat, annot=annotations, fmt="", cmap='coolwarm', square=True, cbar=True, mask=np.isnan(kendall_mat), annot_kws={"size": 12})
        plt.title(f'Kendall Correlation, p-value, and Data Length for {subfolder}')
        plt.savefig(os.path.join(mainfolder, subfolder, folder2, "subjective_objective_corr_timestitch.png"), bbox_inches='tight', dpi=300) 
        plt.show()
    
        #repeating annotations and figure for fdr correction
        #creating a custom annotation matrix combining normality test, tau, p-value, and data length
        annotations_fdr = kendall_mat.copy()
        for meas_1 in df.columns:
            for meas_2 in df.columns:
                if not np.isnan(kendall_mat.loc[meas_1, meas_2]):
                    annotations_fdr.loc[meas_1, meas_2] = f"tau={kendall_mat.loc[meas_1, meas_2]:.2f}\n" #tau
                    if kendall_pval_mat_fdr.loc[meas_1, meas_2] < 0.05:
                        annotations_fdr.loc[meas_1, meas_2] += f"p={kendall_pval_mat_fdr.loc[meas_1, meas_2]:.2f}*\n"
                    else:
                        annotations_fdr.loc[meas_1, meas_2] += f"p={kendall_pval_mat_fdr.loc[meas_1, meas_2]:.2f}\n"
                    annotations_fdr.loc[meas_1, meas_2] += f"dl={int(kendall_len_mat.loc[meas_1, meas_2])}\n"
                    annotations_fdr.loc[meas_1, meas_2] += f"norm={int(shapiro_pval_mat.loc[meas_1, meas_2])}"
                else:
                    annotations_fdr.loc[meas_1, meas_2] = ""
        
        # Plot the combined heatmap with custom annotations (tau, p-value, and data length)
        plt.figure(figsize=(32, 32))
        sns.heatmap(kendall_mat, annot=annotations_fdr, fmt="", cmap='coolwarm', square=True, cbar=True, mask=np.isnan(kendall_mat), annot_kws={"size": 12})
        title = 'Kendall Correlation, p-value (fdr corrected), and Data Length for day: '+ subfolder
        plt.title(title)
        plt.savefig(os.path.join(mainfolder, subfolder, folder2, "subjective_objective_corr_fdr_timestitch.png"), bbox_inches='tight', dpi=300)
        plt.show()
    
        # ========== ADD DATACLOUDS FOR EACH DAY ==========
        print(f"Creating daily datacloud matrix for {subfolder}...")
        #if folder2 is different from what has already been defined, modify and assign it here
        create_daily_datacloud_matrix(df, subfolder, mainfolder, folder2, 
                                     kendall_mat, kendall_pval_mat, kendall_pval_mat_fdr, kendall_len_mat)
        
        #print(f"Creating condensed daily datacloud matrix for {subfolder}...")
        #create_daily_condensed_datacloud_matrix(df, subfolder, mainfolder, folder2,
                                               #kendall_mat, kendall_pval_mat, kendall_pval_mat_fdr, kendall_len_mat)
        
        
        print(f"Completed datacloud analysis for {subfolder}")
        print("-" * 50)



#Next segment - stem plots of tau values with annotated p values for selected measures with focus on subjective stress TET and objective tonic EDA obtained from digital biomarker of wristband

partDates = list(norm_res.keys())  # Or norm_res.keys(), as appropriate
#for asd_001:
#partDates.remove('25_3_24_n17_26_3_24_d')

# List your pairs of interest:
# For example: [('stress', 'eda'), ('stress', 'physical_tension'), ...]
# For each pair to plot
#['dim_1_wakefullness', 'dim_2_boredom', 'dim_3_sensory_avoidance', 'dim_4_social avoidance', 'dim_5_physical tension', 'dim_6_scenario_anxiety', 'dim_7_rumination', 'dim_8_stress', 'dim_9_pain', 'eda', 'pulse_rate', 'pulse_rate_variability', 'resp_rate', 'temp', 'steps', 'acc_std_dev', 'activity', 'met', 'wearing_det']
#adjust as per participant
meas_list = ['dim_1_wakefullness', 'dim_5_physical tension', 'dim_7_rumination', 'dim_8_stress', 'dim_9_pain', 'eda', 'pulse_rate', 'pulse_rate_variability', 'resp_rate', 'acc_std_dev']
measure_pairs = [('dim_8_stress', meas) for meas in meas_list if meas != 'dim_8_stress']
measure_pairs += [('eda', meas) for meas in meas_list if meas != 'eda']
measure_pairs += [('dim_8_stress', 'eda')]

for var1, var2 in measure_pairs:
    tau_vals = []
    p_raw_vals = []
    p_fdr_vals = []
    annotation_texts = []
    tick_labels = []
    valid_day_indices = []

    for idx, day_key in enumerate(partDates):
        try:
            parts = day_key.split('_')
            reqYear = 2000 + int(parts[-2])  #adjust if your data uses 1900-base years - unlikely
            reqMonth = int(parts[-3])
            reqDay = int(parts[-4])
            this_label = giv_date_and_day(reqYear, reqMonth, reqDay)
        except Exception:
            this_label = day_key

        #Retrieve correlation values
        kendall = kendall_mat_dict[day_key]
        pval = kendall_pval_mat_dict[day_key]
        pval_fdr = kendall_pval_mat_fdr_dict[day_key]
        if var1 in kendall.index and var2 in kendall.columns and not np.isnan(kendall.loc[var1, var2]):
            tau = kendall.loc[var1, var2]
            p = pval.loc[var1, var2]
            p_fdr = pval_fdr.loc[var1, var2]
            tau_vals.append(tau)
            p_raw_vals.append(p)
            p_fdr_vals.append(p_fdr)
            if p_fdr < 0.05:
                annotation_texts.append(f"tau={tau:.2f}*\np={p:.2g}\np_fdr={p_fdr:.2g}")
            else:
                annotation_texts.append(f"tau={tau:.2f}\np={p:.2g}\np_fdr={p_fdr:.2g}")
            tick_labels.append(this_label)
            valid_day_indices.append(idx)

    # Now plot only days with valid data
    fig, ax = plt.subplots(figsize=(max(10, len(valid_day_indices)*0.7), 6))
    markerline, stemlines, baseline = ax.stem(range(len(tau_vals)), tau_vals, linefmt='g-', markerfmt='go', basefmt='r-')
    ax.axhline(0, color='red', linestyle='dashed')
    plt.xticks(range(len(tau_vals)), tick_labels, rotation=45, fontsize=10)
    ax.set_ylabel("Correlation (Kendall's tau)")
    ax.set_xlabel("Day (date + weekday)")
    #for graph naming
    if var1.startswith('dim'):
        var1 = var1.split('_')[-1]
    if var2.startswith('dim'):
        var2 = var2.split('_')[-1]
    
    ax.set_title(f"Per-day Correlation: {var1} vs {var2}")

    # Annotate at the top of each stick
    for i, txt in enumerate(annotation_texts):
        ax.text(i, tau_vals[i], txt, ha='left', va='bottom', fontsize=5, color='brown')
   
    plt.tight_layout()
    #specify focus_folder; example folder given below
    focused_folder = r'dataclouds_CAM_LMU\Stream_HC_002\per_day\focused'
    plt.savefig(os.path.join(focused_folder, f"{var1}_{var2}_corr_fdr_timestitch.png"), bbox_inches='tight', dpi=300)
    plt.show()

In [ ]:
"""
SAME AS ABOVE BUT WITH LAGGED CROSS-CORRELATION

#['dim_1_wakefullness', 'dim_2_boredom', 'dim_3_sensory_avoidance', 'dim_4_social avoidance', 'dim_5_physical tension', 'dim_6_scenario_anxiety', 'dim_7_rumination', 'dim_8_stress', 'dim_9_pain', 'eda', 'pulse_rate', 'pulse_rate_variability', 'resp_rate', 'temp', 'steps', 'acc_std_dev', 'activity', 'met', 'wearing_det']

measures_to_pop = ['dim_2_boredom', 'dim_3_sensory_avoidance', 'dim_4_social avoidance', 'dim_6_scenario_anxiety', 'dim_7_rumination','dim_9_pain', 'pulse_rate', 'pulse_rate_variability', 'resp_rate', 'temp', 'steps', 'acc_std_dev', 'activity', 'met', 'wearing_det'] 

#All the instructions of the previous cell apply here. The only difference is the introduction of lags (optional)
Note: Since bins are 15 minytes in length, each hour has 4 bins. Therefore if lag is in the order of hours, specify bins in the corresponding multiples of 4
"""
lag = -24

focus_folder = r'dataclouds_CAM_LMU\Stream_ASD_001\per_day\focused' #specify as per particiipant or if it is the pre-defined folder2
folder_req = os.path.join(focus_folder, 'lag', str(lag))

if not os.path.exists(folder_req):
    os.makedirs(folder_req, exist_ok=True)
focused_folder = folder_req

def giv_date_and_day(reqYear, reqMonth, reqDay):
    date_obj = datetime(reqYear, reqMonth, reqDay)
    return date_obj.strftime("%Y-%m-%d\n%a") 

def lagged_kendall_tau(df, lag):
    # Shift one column by lag (forward lag; negative for backward lag)
    shifted = df.copy()
    col1 = df.columns[0]
    col2 = df.columns[1]
    shifted[col1] = shifted[col1].shift(lag)
    # Drop rows with any NaN (either initial or from shift)
    clean = shifted.dropna(subset=[col1, col2])
    
    stat1, pval1, stat2, pval2, tau, p_value = np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
    if len(clean)>=3:
        stat1, pval1 = shapiro(clean[col1])
        stat2, pval2 = shapiro(clean[col2])
        # Compute Kendall's tau
        tau, p_value = kendalltau(clean[col1], clean[col2])
    return stat1, pval1, stat2, pval2, tau, p_value


def create_daily_datacloud_matrix(df, subfolder, mainfolder, kendall_mat, kendall_pval_mat, kendall_pval_mat_fdr, kendall_len_mat, lag, focused_folder, min_points=1):
    """
    Create a matrix of scatter plots (dataclouds) for daily correlation analysis
    Now uses FDR-corrected p-values for significance
    """
    special_var = 'eda' #modify to dictionary with specified limits as in the previous cell if necessary. Currently this function only looks at eda
    # Filter out columns with all NaN or insufficient data
    valid_columns = []
   
    for col in df.columns:
        if df[col].dropna().shape[0] >= 3:  # Need at least 3 points
            valid_columns.append(col)
    
            
    if len(valid_columns) < 2:
        print(f"Insufficient data for dataclouds on {subfolder}")
        return
    
    n_vars = len(valid_columns)
    
    # Create figure with subplots
    fig, axes = plt.subplots(n_vars, n_vars, figsize=(4*n_vars, 4*n_vars))
    suptitle = f'Daily Datacloud for {subfolder}'
    fig.suptitle(suptitle, fontsize=16, y=0.98)
    
    # If only one variable, make axes 2D for consistent indexing
    if n_vars == 1:
        axes = np.array([[axes]])
    elif n_vars == 2:
        axes = axes.reshape(2, 2)
    
    for i, dim1 in enumerate(valid_columns):
        for j, dim2 in enumerate(valid_columns):
            ax = axes[i, j]
            
            # Get filtered data for this pair
            #filt_data = df[[dim1, dim2]].dropna()
            shifted = df.copy()
            #col1 = df.columns[0]
            #col2 = df.columns[1]
            shifted[dim1] = shifted[dim1].shift(lag)
            # Drop rows with any NaN (either initial or from shift)
            filt_data = shifted.dropna(subset=[dim1, dim2])
            
            if len(filt_data) >= 3:
                if i == j:  # Diagonal - leave blank
                    ax.text(0.5, 0.5, f'Diagonal\n', 
                           transform=ax.transAxes, ha='center', va='center',
                           fontsize=10, color='red')
                    ax.set_xlabel(dim1+'_lag_'+str(lag), fontsize=9)
                    ax.set_ylabel(dim2, fontsize=9)
                else: #if i>j:  # Off-diagonal - show scatter plot #change to just else and show upper triangle too
                    x_data = filt_data[dim1]
                    y_data = filt_data[dim2]
                    # Create scatter plot
                    ax.scatter(x_data, y_data, alpha=0.6, s=20, color='steelblue')
                    if dim1 == special_var: #here is where the upper limit is specified. modify like in previous cell if more variables with different limits are required
                        ax.set_xlim(0, 30)
                    else:
                        ax.set_xlim(0, 4)
                    if dim2 == special_var:
                        ax.set_ylim(0, 30)
                    else:
                        ax.set_ylim(0, 4)
                    # Get correlation statistics from existing matrices
                    if len(filt_data) >= min_points and not np.isnan(kendall_mat.loc[dim1, dim2]):
                        corr_ken = kendall_mat.loc[dim1, dim2]
                        p_ken_raw = kendall_pval_mat.loc[dim1, dim2]
                        p_ken_fdr = kendall_pval_mat_fdr.loc[dim1, dim2]
                        n_points = int(kendall_len_mat.loc[dim1, dim2])
                        
                        # Add trend line if FDR-corrected correlation is significant and reasonably strong
                        if not np.isnan(p_ken_fdr) and p_ken_fdr < 0.05 and abs(corr_ken) > 0.2:
                            # Fit linear trend line
                            z = np.polyfit(x_data, y_data, 1)
                            p = np.poly1d(z)
                            ax.plot(x_data.sort_values(), p(x_data.sort_values()), 
                                   "r--", alpha=0.8, linewidth=1.5)
                        
                        # Add correlation info with both raw and FDR p-values
                        fdr_sig = "*" if (not np.isnan(p_ken_fdr) and p_ken_fdr < 0.05) else ""
                        raw_sig = "*" if p_ken_raw < 0.05 else ""
                        
                        annotation_text = f'tau={corr_ken:.3f}{fdr_sig}\n'
                        annotation_text += f'p_raw={p_ken_raw:.3f}{raw_sig}\n'
                        if not np.isnan(p_ken_fdr):
                            annotation_text += f'p_fdr={p_ken_fdr:.3f}{fdr_sig}\n'
                        else:
                            annotation_text += f'p_fdr=n/a\n'
                        annotation_text += f'n={n_points}'
                        
                        ax.text(0.05, 0.95, annotation_text,
                               transform=ax.transAxes, fontsize=7,
                               verticalalignment='top',
                               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
                    
                    ax.set_xlabel(dim1+'_lag_'+str(lag), fontsize=9)
                    ax.set_ylabel(dim2, fontsize=9)
                
                    
            else:
                # Not enough data
                ax.text(0.5, 0.5, f'Insufficient data\n(n={len(filt_data)})', 
                       transform=ax.transAxes, ha='center', va='center',
                       fontsize=10, color='red')
                ax.set_xlabel(dim1+'_lag_'+str(lag), fontsize=9)
                ax.set_ylabel(dim2, fontsize=9)
                if dim1 == special_var:
                    ax.set_xlim(0, 30)
                else:
                    ax.set_xlim(0, 4)
                if dim2 == special_var:
                    ax.set_ylim(0, 30)
                else:
                    ax.set_ylim(0, 4)
            
            # Clean up axes
            ax.tick_params(labelsize=8)
            ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    
    plt.savefig(os.path.join(focused_folder, f"focused_daily_datacloud_fdr_{subfolder}_timestitch.png"),bbox_inches='tight', dpi=300)
    plt.show()


def create_daily_condensed_datacloud_matrix(df, subfolder, mainfolder, folder2, kendall_mat, kendall_pval_mat, kendall_pval_mat_fdr, kendall_len_mat, min_points=1):
    """
    Create a condensed matrix showing only significant daily correlations (lower triangle)
    Now uses FDR-corrected p-values for significance
    """
    # Filter out columns with all NaN or insufficient data
    valid_columns = []
    for col in df.columns:
        if df[col].dropna().shape[0] >= 3:
            valid_columns.append(col)
    
    if len(valid_columns) < 2:
        print(f"Need at least 2 variables for correlation dataclouds on {subfolder}")
        return
    
    # Calculate number of unique pairs
    n_vars = len(valid_columns)
    n_unique_pairs = n_vars * (n_vars - 1) // 2
    
    if n_unique_pairs == 0:
        print(f"Need at least 2 variables for correlation dataclouds on {subfolder}")
        return
    
    # Determine subplot layout
    cols = min(4, n_unique_pairs)  # Max 4 columns
    rows = (n_unique_pairs + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 4*rows))
    suptitle = f'Daily Significant Correlations (FDR-corrected) - Dataclouds for {subfolder}'
    fig.suptitle(suptitle, fontsize=14)
    
    # Make axes iterable
    if n_unique_pairs == 1:
        axes = [axes]
    else:
        axes = axes.flatten() if hasattr(axes, 'flatten') else axes
    
    plot_idx = 0
    significant_pairs = []
    
    # Collect all pairs first (lower triangle)
    for i, dim1 in enumerate(valid_columns):
        for j, dim2 in enumerate(valid_columns):
            if i <= j:  # Only lower triangle
                continue
                
            # Get filtered data
            filt_data = df[[dim1, dim2]].dropna()
            
            if len(filt_data) >= min_points and not np.isnan(kendall_mat.loc[dim1, dim2]):
                corr_ken = kendall_mat.loc[dim1, dim2]
                p_ken_raw = kendall_pval_mat.loc[dim1, dim2]
                p_ken_fdr = kendall_pval_mat_fdr.loc[dim1, dim2]
                n_points = int(kendall_len_mat.loc[dim1, dim2])
                significant_pairs.append((dim1, dim2, filt_data, corr_ken, p_ken_raw, p_ken_fdr, n_points))
    
    # Sort by absolute correlation strength
    significant_pairs.sort(key=lambda x: abs(x[3]), reverse=True)
    
    # Plot significant pairs
    for dim1, dim2, filt_data, corr_ken, p_ken_raw, p_ken_fdr, n_points in significant_pairs[:len(axes)]:
        ax = axes[plot_idx]
        
        x_data = filt_data[dim1]
        y_data = filt_data[dim2]
        
        # Create scatter plot
        ax.scatter(x_data, y_data, alpha=0.6, s=30, color='steelblue')
        
        # Add trend line for FDR-significant correlations
        if not np.isnan(p_ken_fdr) and p_ken_fdr < 0.05:
            z = np.polyfit(x_data, y_data, 1)
            p = np.poly1d(z)
            x_trend = np.linspace(x_data.min(), x_data.max(), 100)
            ax.plot(x_trend, p(x_trend), "r-", alpha=0.8, linewidth=2)
        
        # Add statistics with FDR information
        fdr_significance = "*" if (not np.isnan(p_ken_fdr) and p_ken_fdr < 0.05) else ""
        raw_significance = "*" if p_ken_raw < 0.05 else ""
        
        title_text = f'{dim1} vs {dim2}\ntau={corr_ken:.3f}{fdr_significance}'
        subtitle_text = f'p_raw={p_ken_raw:.3f}{raw_significance}'
        if not np.isnan(p_ken_fdr):
            subtitle_text += f', p_fdr={p_ken_fdr:.3f}{fdr_significance}'
        subtitle_text += f', n={n_points}'
        
        ax.set_title(title_text, fontsize=10, fontweight='bold')
        ax.text(0.5, -0.15, subtitle_text, transform=ax.transAxes, 
                ha='center', fontsize=8, style='italic')
        
        ax.set_xlabel(dim1, fontsize=9)
        ax.set_ylabel(dim2, fontsize=9)
        ax.grid(True, alpha=0.3)
        
        plot_idx += 1
    
    # Hide unused subplots
    for idx in range(plot_idx, len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    plt.savefig(os.path.join(mainfolder, subfolder, folder2, f"filtered_daily_significant_dataclouds_fdr_{subfolder}_timestitch.png"), bbox_inches='tight', dpi=300)
    plt.show()
    return
    
   



# Main per-day analysis loop with integrated dataclouds
kendall_mat_dict = {}
kendall_pval_mat_dict = {}
kendall_pval_mat_fdr_dict = {}
kendall_len_mat_dict = {}
shapiro_pval_mat_dict = {}



for subfolder in norm_res.keys():
    if subfolder == '25_3_24_n17_26_3_24_d': #instruction same as previous cell
        continue
    else:
        dim_dict = {}
        for dim_qx in norm_res[subfolder].keys():
            if dim_qx in measures_to_pop:
                continue
                
            dim_dict[dim_qx] = norm_res[subfolder][dim_qx]['full data']
        df = pd.DataFrame(dim_dict) 
        #now need a corellogram such that for values =-5000 are turned to NaN and dropped only for the two columns being correlated. Kendall correlation using scipy needs to be done
        df.replace(-5000, np.nan, inplace=True)
        #shapiro_mat = pd.DataFrame(np.zeros((df.shape[1], df.shape[1])), columns=df.columns, index=df.columns)
        shapiro_pval_mat = pd.DataFrame(np.zeros((df.shape[1], df.shape[1])), columns=df.columns, index=df.columns)
        kendall_mat = pd.DataFrame(np.zeros((df.shape[1], df.shape[1])), columns=df.columns, index=df.columns)
        kendall_pval_mat = pd.DataFrame(np.zeros((df.shape[1], df.shape[1])), columns=df.columns, index=df.columns)
        kendall_len_mat = pd.DataFrame(np.zeros((df.shape[1], df.shape[1])), columns=df.columns, index=df.columns)
    
        #for fdr correction
        kendall_pval_mat_fdr = pd.DataFrame(np.zeros((df.shape[1], df.shape[1])), columns=df.columns, index=df.columns)
        
        
        for dim1 in df.columns:
            for dim2 in df.columns:
                filt_data = df[[dim1, dim2]] #.dropna() dropping nans after introducing lags
                stat1, pval1, stat2, pval2, corr_ken, p_ken = lagged_kendall_tau(filt_data, lag)
                
    
                if corr_ken!= np.nan:
                    if pval1 > 0.05 and pval2 > 0.05:
                        shapiro_pval_mat.loc[dim1, dim2] = 1
                    else:
                        shapiro_pval_mat.loc[dim1, dim2] = 0
                    kendall_mat.loc[dim1, dim2] = corr_ken
                    kendall_pval_mat.loc[dim1,dim2] = p_ken
                    kendall_len_mat.loc[dim1,dim2] = len(filt_data)
                else:
                    #not enough data to test normality or calculate kendall corr
                    shapiro_pval_mat.loc[dim1, dim2] = np.nan
                    kendall_mat.loc[dim1, dim2] = np.nan
                    kendall_pval_mat.loc[dim1,dim2] = np.nan
                    kendall_len_mat.loc[dim1,dim2] = len(filt_data)
    
        #Applying fdr correction
        temp_store = []
        pPreFdr = []
        pos = []
        for meas_1 in df.columns:
            temp_store.append(meas_1)
            for meas_2 in df.columns:
                if meas_1 == meas_2:
                    continue
                elif meas_2 in temp_store:
                    continue
                else:
                    if not np.isnan(kendall_mat.loc[meas_1, meas_2]):
                        pPreFdr.append(kendall_pval_mat.loc[meas_1, meas_2])
                        pos.append((meas_1, meas_2))
        
        pPostFdr = multipletests(pPreFdr, method='fdr_bh')[1]
        
        for (meas_1, meas_2), p_adj in zip(pos, pPostFdr):
            kendall_pval_mat_fdr.loc[meas_1, meas_2] = p_adj
            kendall_pval_mat_fdr.loc[meas_2, meas_1] = p_adj
        
        #fill diagonal and any missing values with NaN 
        for meas in df.columns:
            kendall_pval_mat_fdr.loc[meas, meas] = np.nan
    
        #storing values per day
        kendall_mat_dict[subfolder] = kendall_mat.copy()
        kendall_pval_mat_dict[subfolder] = kendall_pval_mat.copy()
        kendall_pval_mat_fdr_dict[subfolder] = kendall_pval_mat_fdr.copy()
        kendall_len_mat_dict[subfolder] = kendall_len_mat.copy()
        shapiro_pval_mat_dict[subfolder] = shapiro_pval_mat.copy()
    
        #creating a custom annotation matrix combining tau, p-value, and data length
        annotations = kendall_mat.copy()
        
        for dim1 in df.columns:
            for dim2 in df.columns:
                if not np.isnan(kendall_mat.loc[dim1, dim2]):
                    annotations.loc[dim1, dim2] = f"tau={kendall_mat.loc[dim1, dim2]:.2f}\n"
                    if kendall_pval_mat.loc[dim1, dim2] < 0.05:
                        annotations.loc[dim1, dim2] += f"p={kendall_pval_mat.loc[dim1, dim2]:.2f}*\n"
                    else:
                        annotations.loc[dim1, dim2] += f"p={kendall_pval_mat.loc[dim1, dim2]:.2f}\n"
                    annotations.loc[dim1, dim2] += f"dl={int(kendall_len_mat.loc[dim1, dim2])}\n"
                    annotations.loc[dim1, dim2] += f"norm={int(shapiro_pval_mat.loc[dim1, dim2])}\n"
                    annotations.loc[dim1, dim2] += f"lag_dim={dim1}"
                else:
                    annotations.loc[dim1, dim2] = ""
    
        # Plot the combined heatmap with custom annotations (tau, p-value, and data length)
        plt.figure(figsize=(32, 32))
        sns.heatmap(kendall_mat, annot=annotations, fmt="", cmap='coolwarm', square=True, cbar=True, mask=np.isnan(kendall_mat), annot_kws={"size": 12})
        plt.title(f'Kendall Correlation, p-value, and Data Length for {subfolder}')
        plt.savefig(os.path.join(mainfolder, subfolder, folder2, "subjective_objective_corr_timestitch.png"), bbox_inches='tight', dpi=300)
        plt.show()
    
        #repeating annotations and figure for fdr correction
        #creating a custom annotation matrix combining normality test, tau, p-value, and data length
        annotations_fdr = kendall_mat.copy()
        for meas_1 in df.columns:
            for meas_2 in df.columns:
                if not np.isnan(kendall_mat.loc[meas_1, meas_2]):
                    annotations_fdr.loc[meas_1, meas_2] = f"tau={kendall_mat.loc[meas_1, meas_2]:.2f}\n" #tau
                    if kendall_pval_mat_fdr.loc[meas_1, meas_2] < 0.05:
                        annotations_fdr.loc[meas_1, meas_2] += f"p={kendall_pval_mat_fdr.loc[meas_1, meas_2]:.2f}*\n"
                    else:
                        annotations_fdr.loc[meas_1, meas_2] += f"p={kendall_pval_mat_fdr.loc[meas_1, meas_2]:.2f}\n"
                    annotations_fdr.loc[meas_1, meas_2] += f"dl={int(kendall_len_mat.loc[meas_1, meas_2])}\n"
                    annotations_fdr.loc[meas_1, meas_2] += f"norm={int(shapiro_pval_mat.loc[meas_1, meas_2])}\n"
                    annotations_fdr.loc[meas_1, meas_2] += f"lag_dim={meas_1}"
                else:
                    annotations_fdr.loc[meas_1, meas_2] = ""
        
        # Plot the combined heatmap with custom annotations (tau, p-value, and data length)
        plt.figure(figsize=(32, 32))
        sns.heatmap(kendall_mat, annot=annotations_fdr, fmt="", cmap='coolwarm', square=True, cbar=True, mask=np.isnan(kendall_mat), annot_kws={"size": 12})
        title = 'Kendall Correlation, p-value (fdr corrected), and Data Length for day: '+ subfolder
        plt.title(title)
        plt.savefig(os.path.join(mainfolder, subfolder, folder2, "subjective_objective_corr_fdr_timestitch.png"), bbox_inches='tight', dpi=300)
        plt.show()
    
        # ========== ADD DATACLOUDS FOR EACH DAY ==========
        print(f"Creating daily datacloud matrix for {subfolder}...")
        create_daily_datacloud_matrix(df, subfolder, mainfolder,
                                     kendall_mat, kendall_pval_mat, kendall_pval_mat_fdr, kendall_len_mat, lag, focused_folder)
        
        #print(f"Creating condensed daily datacloud matrix for {subfolder}...")
        #create_daily_condensed_datacloud_matrix(df, subfolder, mainfolder, folder2,
                                               #kendall_mat, kendall_pval_mat, kendall_pval_mat_fdr, kendall_len_mat)
        
        
        print(f"Completed datacloud analysis for {subfolder}")
        print("-" * 50)

#Same as previous cell

partDates = list(norm_res.keys())  # Or norm_res.keys(), as appropriate
#for asd_001: (modify as per participant and analysis needs)
#partDates.remove('25_3_24_n17_26_3_24_d')

#['dim_1_wakefullness', 'dim_2_boredom', 'dim_3_sensory_avoidance', 'dim_4_social avoidance', 'dim_5_physical tension', 'dim_6_scenario_anxiety', 'dim_7_rumination', 'dim_8_stress', 'dim_9_pain', 'eda', 'pulse_rate', 'pulse_rate_variability', 'resp_rate', 'temp', 'steps', 'acc_std_dev', 'activity', 'met', 'wearing_det']
#(modify as per participant and analysis needs)
meas_list = ['dim_1_wakefullness', 'dim_5_physical tension', 'dim_8_stress', 'eda'] 
measure_pairs = [('dim_8_stress', meas) for meas in meas_list if meas != 'dim_8_stress']
measure_pairs += [('eda', meas) for meas in meas_list if meas != 'eda']
measure_pairs += [('dim_8_stress', 'eda')]
measure_pairs += [('dim_1_wakefullness', meas) for meas in meas_list if meas != 'dim_1_wakefullness']
measure_pairs += [('dim_5_physical tension', meas) for meas in meas_list if meas != 'dim_5_physical tension']

for var1, var2 in measure_pairs:
    tau_vals = []
    p_raw_vals = []
    p_fdr_vals = []
    annotation_texts = []
    tick_labels = []
    valid_day_indices = []

    for idx, day_key in enumerate(partDates):
        try:
            parts = day_key.split('_')
            reqYear = 2000 + int(parts[-2])  # adjust if your data uses 1900-base years - unlikely
            reqMonth = int(parts[-3])
            reqDay = int(parts[-4])
            this_label = giv_date_and_day(reqYear, reqMonth, reqDay)
        except Exception:
            this_label = day_key

        # Retrieve correlation values
        kendall = kendall_mat_dict[day_key]
        pval = kendall_pval_mat_dict[day_key]
        pval_fdr = kendall_pval_mat_fdr_dict[day_key]
        if var1 in kendall.index and var2 in kendall.columns and not np.isnan(kendall.loc[var1, var2]):
            tau = kendall.loc[var1, var2]
            p = pval.loc[var1, var2]
            p_fdr = pval_fdr.loc[var1, var2]
            tau_vals.append(tau)
            p_raw_vals.append(p)
            p_fdr_vals.append(p_fdr)
            if p_fdr < 0.05:
                annotation_texts.append(f"tau={tau:.2f}*\np={p:.2g}\np_fdr={p_fdr:.2g}")
            else:
                annotation_texts.append(f"tau={tau:.2f}\np={p:.2g}\np_fdr={p_fdr:.2g}")
            tick_labels.append(this_label)
            valid_day_indices.append(idx)

    # Now plot only days with valid data
    fig, ax = plt.subplots(figsize=(max(10, len(valid_day_indices)*0.7), 6))
    markerline, stemlines, baseline = ax.stem(range(len(tau_vals)), tau_vals, linefmt='g-', markerfmt='go', basefmt='r-')
    ax.axhline(0, color='red', linestyle='dashed')
    plt.xticks(range(len(tau_vals)), tick_labels, rotation=45, fontsize=10)
    ax.set_ylabel("Correlation (Kendall's tau)")
    ax.set_xlabel("Day (date + weekday)")
    #for graph naming
    var1 = var1.split('_')[-1]
    var2 = var2.split('_')[-1]
    
    ax.set_title(f"Per-day Correlation: {var1} (lagged) vs {var2}")

    # Annotate at the top of each stick
    for i, txt in enumerate(annotation_texts):
        ax.text(i, tau_vals[i], txt, ha='left', va='bottom', fontsize=5, color='brown')
    
    plt.tight_layout()
    
    plt.savefig(os.path.join(focused_folder, f"{var1}_{var2}_corr_fdr_timestitch.png"), bbox_inches='tight', dpi=300)
    plt.show()

In [ ]:
#NOW FOR CORRELATIONS OF ALL DAYS TOGETHER TO GET A SENSE OF OVERALL TRENDS BETWEEN MEASURES

In [ ]:
sublist = list(dim_q.keys())
sublist

In [ ]:
oblist = list(dict_meas.keys())
oblist

In [ ]:
subOb = sublist+oblist
subOb

In [ ]:
subOb_dict = dim_q | dict_meas
subOb_dict

In [ ]:
subOb_dict.keys()

In [ ]:
"""
ALL DAY CORELLOGRAM
"""

kendall_mat_all_day = pd.DataFrame(np.zeros((len(subOb), len(subOb))), columns=subOb, index=subOb)
kendall_mat_all_day

In [ ]:
#OPTIONAL: If not all measures need to be analysed, list the the ones to exclude below (example shown). If not, skip cell
measures_to_pop = ['dim_2_boredom', 'dim_3_sensory_avoidance', 'dim_4_social avoidance', 'dim_6_scenario_anxiety', 'temp', 'steps', 'activity', 'met', 'wearing_det'] 

subOb = [x for x in subOb if x not in measures_to_pop]

for key in measures_to_pop:
    subOb_dict.pop(key, None)



In [ ]:
#OPTIONAL: If some days have to be excluded, list them below and execute cell. If not, skip
days_to_pop = ['08_3_24_n1_9_3_24_d', '11_3_24_n3_12_3_24_d', '12_3_24_n4_13_3_24_d', '23_3_24_n15_24_3_24_d', '24_3_24_n16_25_3_24_d', '25_3_24_n17_26_3_24_d']
"""
for meas in subOb_dict.keys():
    #print(day)
    for day in subOb_dict[meas].keys():
        if day in days_to_pop:
            subOb_dict[meas].pop(day, None)
"""
for meas in subOb_dict.keys():
    for day in list(subOb_dict[meas].keys()):  # make a list copy
        if day in days_to_pop:
            subOb_dict[meas].pop(day, None)

In [ ]:
"""
JUST LIKE THE PER-DAY VERSION IN PREVIOUS CELLS
"""


shapiro_mat_all_day = pd.DataFrame(np.zeros((len(subOb), len(subOb))), columns=subOb, index=subOb)
shapiro_pval_mat_all_day = pd.DataFrame(np.zeros((len(subOb), len(subOb))), columns=subOb, index=subOb)
#for correlation test
kendall_mat_all_day = pd.DataFrame(np.zeros((len(subOb), len(subOb))), columns=subOb, index=subOb)
kendall_pval_mat_all_day = pd.DataFrame(np.zeros((len(subOb), len(subOb))), columns=subOb, index=subOb)
kendall_len_mat_all_day = pd.DataFrame(np.zeros((len(subOb), len(subOb))), columns=subOb, index=subOb)
#for fdr correction
kendall_pval_mat_all_day_fdr = pd.DataFrame(np.zeros((len(subOb), len(subOb))), columns=subOb, index=subOb)


for dim_qx in subOb_dict:
    for dim_qy in subOb_dict:
        dimqx = []
        dimqy = []
        for subfolder in subOb_dict[dim_qx].keys():
            if subfolder in subOb_dict[dim_qy].keys():
                #print(subfolder)
                #print(f'Appending data for common subfolder: {subfolder} for dims {dim_qx} and {dim_qy}')
                for val_x in subOb_dict[dim_qx][subfolder].values():
                    dimqx.append(val_x)
                for val_y in subOb_dict[dim_qy][subfolder].values():
                    dimqy.append(val_y)

        df = pd.DataFrame({f'{dim_qx}_1' : dimqx, f'{dim_qy}_2' : dimqy}) 
        #filter the data by turning -5000 values to nan and then dropping the nans
        df.replace(-5000, np.nan, inplace=True)
        filt_data = df.dropna()
        if len(filt_data)>=2:
                stat1, pval1 = shapiro(filt_data[f'{dim_qx}_1'])
                stat2, pval2 = shapiro(filt_data[f'{dim_qy}_2'])
                corr_ken, p_ken = kendalltau(filt_data[f'{dim_qx}_1'], filt_data[f'{dim_qy}_2'])
                if pval1 > 0.05 and pval2 > 0.05:
                    shapiro_pval_mat_all_day.loc[dim_qx, dim_qy] = 1
                else:
                    shapiro_pval_mat_all_day.loc[dim_qx, dim_qy] = 0
                kendall_mat_all_day.loc[dim_qx, dim_qy] = corr_ken
                kendall_pval_mat_all_day.loc[dim_qx, dim_qy] = p_ken
                kendall_len_mat_all_day.loc[dim_qx, dim_qy] = len(filt_data)
        else:
                #not enough data to test normality or calculate kendall corr
                shapiro_pval_mat_all_day.loc[dim_qx, dim_qy] = np.nan
                kendall_mat_all_day.loc[dim_qx, dim_qy] = np.nan
                kendall_pval_mat_all_day.loc[dim_qx, dim_qy] = np.nan
                kendall_len_mat_all_day.loc[dim_qx, dim_qy] = len(filt_data)

#Applying fdr correction
#
temp_store = []
pPreFdr = []
pos = []
for meas_1 in subOb_dict:
    temp_store.append(meas_1)
    for meas_2 in subOb_dict:
        if meas_1 == meas_2:
            continue
        elif meas_2 in temp_store:
            continue
        else:
            if not np.isnan(kendall_mat_all_day.loc[meas_1, meas_2]):
                pPreFdr.append(kendall_pval_mat_all_day.loc[meas_1, meas_2])
                pos.append((meas_1, meas_2))

pPostFdr = multipletests(pPreFdr, method='fdr_bh')[1]

for (meas_1, meas_2), p_adj in zip(pos, pPostFdr):
    kendall_pval_mat_all_day_fdr.loc[meas_1, meas_2] = p_adj
    kendall_pval_mat_all_day_fdr.loc[meas_2, meas_1] = p_adj

#fill diagonal and any missing values with NaN 
for meas in subOb_dict:
    kendall_pval_mat_all_day_fdr.loc[meas, meas] = np.nan

#creating a custom annotation matrix combining normality test, tau, p-value, and data length
annotations = kendall_mat_all_day.copy()
for dim_qx in subOb_dict:
    for dim_qy in subOb_dict:
        if not np.isnan(kendall_mat_all_day.loc[dim_qx, dim_qy]):
            annotations.loc[dim_qx, dim_qy] = f"tau={kendall_mat_all_day.loc[dim_qx, dim_qy]:.2f}\n"
            if kendall_pval_mat_all_day.loc[dim_qx, dim_qy] < 0.05:
                annotations.loc[dim_qx, dim_qy] += f"p={kendall_pval_mat_all_day.loc[dim_qx, dim_qy]:.3f}*\n"
            else:
                annotations.loc[dim_qx, dim_qy] += f"p={kendall_pval_mat_all_day.loc[dim_qx, dim_qy]:.3f}\n"
            annotations.loc[dim_qx, dim_qy] += f"dl={int(kendall_len_mat_all_day.loc[dim_qx, dim_qy])}\n"
            annotations.loc[dim_qx, dim_qy] += f"norm={int(shapiro_pval_mat_all_day.loc[dim_qx, dim_qy])}"
        else:
            annotations.loc[dim_qx, dim_qy] = ""

# Plot the combined heatmap with custom annotations (tau, p-value, and data length)
plt.figure(figsize=(32, 32))
sns.heatmap(kendall_mat_all_day, annot=annotations, fmt="", cmap='coolwarm', square=True, cbar=True, mask=np.isnan(kendall_mat_all_day), annot_kws={"size": 12})
title = 'Kendall Correlation, p-value, and Data Length for participant: '+ mainfolder.split("\\")[-1]
plt.title(title)
plt.savefig(os.path.join(mainfolder, "all_day_subjective_objective_correlogram_fifteen_min_timestitch.png"), bbox_inches='tight', dpi=300)
plt.show()

#repeating annotations and figure for fdr correction
annotations_fdr = kendall_mat_all_day.copy()
for meas_1 in subOb_dict:
    for meas_2 in subOb_dict:
        if not np.isnan(kendall_mat_all_day.loc[meas_1, meas_2]):
            annotations_fdr.loc[meas_1, meas_2] = f"tau={kendall_mat_all_day.loc[meas_1, meas_2]:.2f}\n"
            if kendall_pval_mat_all_day_fdr.loc[meas_1, meas_2] < 0.05:
                annotations_fdr.loc[meas_1, meas_2] += f"p={kendall_pval_mat_all_day_fdr.loc[meas_1, meas_2]:.2f}*\n"
            else:
                annotations_fdr.loc[meas_1, meas_2] += f"p={kendall_pval_mat_all_day_fdr.loc[meas_1, meas_2]:.2f}\n"
            annotations_fdr.loc[meas_1, meas_2] += f"dl={int(kendall_len_mat_all_day.loc[meas_1, meas_2])}\n"
            annotations_fdr.loc[meas_1, meas_2] += f"norm={int(shapiro_pval_mat_all_day.loc[meas_1, meas_2])}"
        else:
            annotations_fdr.loc[meas_1, meas_2] = ""

# Plot the combined heatmap with custom annotations (tau, p-value, and data length)
plt.figure(figsize=(32, 32))
sns.heatmap(kendall_mat_all_day, annot=annotations_fdr, fmt="", cmap='coolwarm', square=True, cbar=True, mask=np.isnan(kendall_mat_all_day), annot_kws={"size": 12})
title = 'Kendall Correlation, p-value (fdr corrected), and Data Length for participant: '+ mainfolder.split("\\")[-1]
plt.title(title)
plt.savefig(os.path.join(mainfolder, "all_day_subjective_objective_correlogram_fifteen_min_fdr_timestitch.png"), bbox_inches='tight', dpi=300)
plt.show()

#trying out dataclouds with FDR correction

def create_datacloud_matrix(subOb_dict, subOb, mainfolder, kendall_mat_all_day, kendall_pval_mat_all_day, kendall_pval_mat_all_day_fdr, kendall_len_mat_all_day, min_points=10):
    """
    Create a matrix of scatter plots (dataclouds) corresponding to correlation analysis
    Now uses FDR-corrected p-values for significance
    """
    special_var = {'eda': 30, 'pulse_rate':150, 'pulse_rate_variability':300, 'resp_rate':30, 'acc_std_dev':0.5}  #specify limits as required
    n_vars = len(subOb)
    
    # Create figure with subplots
    fig, axes = plt.subplots(n_vars, n_vars, figsize=(3*n_vars, 2*n_vars))
    participant_title_list = mainfolder.split("\\")[-1].split('_')[0:3]
    participant_title = '_'.join(participant_title_list)
    suptitle = 'Datacloud Matrix (FDR-corrected) for participant: '+ participant_title
    fig.suptitle(suptitle, fontsize=26, y=0.98)
    
    # If only one variable, make axes 2D for consistent indexing
    if n_vars == 1:
        axes = np.array([[axes]])
    elif n_vars == 2:
        axes = axes.reshape(2, 2)
    
    for i, dim_qx in enumerate(subOb):
        for j, dim_qy in enumerate(subOb):
            ax = axes[i, j]
            
            # Collect data for this pair
            dimqx = []
            dimqy = []
            
            for subfolder in subOb_dict[dim_qx].keys():
                if subfolder in subOb_dict[dim_qy].keys():
                    for val_x in subOb_dict[dim_qx][subfolder].values():
                        dimqx.append(val_x)
                    for val_y in subOb_dict[dim_qy][subfolder].values():
                        dimqy.append(val_y)
            
            # Create DataFrame and filter data
            df = pd.DataFrame({f'{dim_qx}_1': dimqx, f'{dim_qy}_2': dimqy})
            df.replace(-5000, np.nan, inplace=True)
            filt_data = df.dropna()
            
            if len(filt_data) >= 2:
                x_data = filt_data[f'{dim_qx}_1']
                y_data = filt_data[f'{dim_qy}_2']
                
                if i == j:  # Diagonal - leave blank
                    ax.text(0.5, 0.5, f'Diagonal', 
                           transform=ax.transAxes, ha='center', va='center',
                           fontsize=10, color='red')
                    ax.set_xlabel(dim_qx, fontsize=9)
                    ax.set_ylabel(dim_qy, fontsize=9)
                elif i>j:  # Off-diagonal - show scatter plot
                    # Create scatter plot
                    ax.scatter(x_data, y_data, alpha=0.6, s=20, color='steelblue')
                    if dim_qx in special_var.keys():
                        for key, val in special_var.items():
                            if dim_qx == key:
                                ax.set_xlim(0, val)
                    else:
                        ax.set_xlim(0, 4)
                    if dim_qy in special_var.keys():
                        for key, val in special_var.items():
                            if dim_qy == key:
                                ax.set_ylim(0, val)
                    else:
                        ax.set_ylim(0, 4)
                    
                    # Get correlation statistics from existing matrices
                    if len(filt_data) >= min_points and not np.isnan(kendall_mat_all_day.loc[dim_qx, dim_qy]):
                        corr_ken = kendall_mat_all_day.loc[dim_qx, dim_qy]
                        p_ken_raw = kendall_pval_mat_all_day.loc[dim_qx, dim_qy]
                        p_ken_fdr = kendall_pval_mat_all_day_fdr.loc[dim_qx, dim_qy]
                        n_points = int(kendall_len_mat_all_day.loc[dim_qx, dim_qy])
                        
                        # Add trend line if FDR-corrected correlation is significant and reasonably strong
                        if not np.isnan(p_ken_fdr) and p_ken_fdr < 0.05 : #and abs(corr_ken) > 0.2
                            # Fit linear trend line
                            z = np.polyfit(x_data, y_data, 1)
                            p = np.poly1d(z)
                            ax.plot(x_data.sort_values(), p(x_data.sort_values()), 
                                   "r--", alpha=0.8, linewidth=1.5)
                        
                        # Add correlation info with both raw and FDR p-values
                        fdr_sig = "*" if (not np.isnan(p_ken_fdr) and p_ken_fdr < 0.05) else ""
                        raw_sig = "*" if p_ken_raw < 0.05 else ""
                        
                        annotation_text = f'tau={corr_ken:.3f}{fdr_sig}\n'
                        annotation_text += f'p_raw={p_ken_raw:.3f}{raw_sig}\n'
                        if not np.isnan(p_ken_fdr):
                            annotation_text += f'p_fdr={p_ken_fdr:.3f}{fdr_sig}\n'
                        else:
                            annotation_text += f'p_fdr=n/a\n'
                        annotation_text += f'n={n_points}'
                        
                        ax.text(0.05, 0.95, annotation_text,
                               transform=ax.transAxes, fontsize=7,
                               verticalalignment='top',
                               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
                    
                    ax.set_xlabel(dim_qx, fontsize=9)
                    ax.set_ylabel(dim_qy, fontsize=9)
                else:
                    #upper triangle
                    ax.text(0.5, 0.5, f'Upper triangle', 
                           transform=ax.transAxes, ha='center', va='center',
                           fontsize=10, color='red')
                    ax.set_xlabel(dim_qx, fontsize=9)
                    ax.set_ylabel(dim_qy, fontsize=9)
                    if dim_qx == special_var:
                        ax.set_xlim(0, 30)
                    else:
                        ax.set_xlim(0, 4)
                    if dim_qy == special_var:
                        ax.set_ylim(0, 30)
                    else:
                        ax.set_ylim(0, 4)
            else:
                # Not enough data
                ax.text(0.5, 0.5, f'Insufficient data\n(n={len(filt_data)})', 
                       transform=ax.transAxes, ha='center', va='center',
                       fontsize=10, color='red')
                ax.set_xlabel(dim_qx, fontsize=9)
                ax.set_ylabel(dim_qy, fontsize=9)
                if dim_qx == special_var:
                    ax.set_xlim(0, 30)
                else:
                    ax.set_xlim(0, 4)
                if dim_qy == special_var:
                    ax.set_ylim(0, 30)
                else:
                    ax.set_ylim(0, 4)
            
            # Clean up axes
            ax.tick_params(labelsize=8)
            ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    focus_folder = r'dataclouds_CAM_LMU\Stream_ASD_001\all_days' #specify as required or also change it to mainfolder
    plt.savefig(os.path.join(focus_folder, "focused_datacloud_matrix_fdr_fifteen_min_timestitch.png"), bbox_inches='tight', dpi=300)
    plt.show()


def create_condensed_datacloud_matrix(subOb_dict, subOb, mainfolder, kendall_mat_all_day, kendall_pval_mat_all_day, kendall_pval_mat_all_day_fdr, kendall_len_mat_all_day, min_points=10):
    """
    In descending order correlation strength
    Create a condensed matrix showing only unique pairs (lower triangle)
    Now uses FDR-corrected p-values for significance
    """
    # Calculate number of unique pairs
    n_vars = len(subOb)
    n_unique_pairs = n_vars * (n_vars - 1) // 2
    
    if n_unique_pairs == 0:
        print("Need at least 2 variables for correlation dataclouds")
        return
    
    # Determine subplot layout
    cols = min(5, n_unique_pairs)  # Max 5 columns
    rows = (n_unique_pairs + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 4*rows))
    suptitle = 'Significant Correlations (FDR-corrected) - Dataclouds for participant: '+mainfolder.split("\\")[-1]
    fig.suptitle(suptitle, fontsize=14)
    
    # Make axes iterable
    if n_unique_pairs == 1:
        axes = [axes]
    else:
        axes = axes.flatten() if hasattr(axes, 'flatten') else axes
    
    plot_idx = 0
    significant_pairs = []
    
    # Collect all pairs first (lower triangle)
    for i, dim_qx in enumerate(subOb):
        for j, dim_qy in enumerate(subOb):
            if i <= j:  # Only lower triangle
                continue
                
            # Collect and filter data
            dimqx = []
            dimqy = []
            
            for subfolder in subOb_dict[dim_qx].keys():
                if subfolder in subOb_dict[dim_qy].keys():
                    for val_x in subOb_dict[dim_qx][subfolder].values():
                        dimqx.append(val_x)
                    for val_y in subOb_dict[dim_qy][subfolder].values():
                        dimqy.append(val_y)
            
            df = pd.DataFrame({f'{dim_qx}_1': dimqx, f'{dim_qy}_2': dimqy})
            df.replace(-5000, np.nan, inplace=True)
            filt_data = df.dropna()
            
            if len(filt_data) >= min_points and not np.isnan(kendall_mat_all_day.loc[dim_qx, dim_qy]):
                corr_ken = kendall_mat_all_day.loc[dim_qx, dim_qy]
                p_ken_raw = kendall_pval_mat_all_day.loc[dim_qx, dim_qy]
                p_ken_fdr = kendall_pval_mat_all_day_fdr.loc[dim_qx, dim_qy]
                n_points = int(kendall_len_mat_all_day.loc[dim_qx, dim_qy])
                significant_pairs.append((dim_qx, dim_qy, filt_data, corr_ken, p_ken_raw, p_ken_fdr, n_points))
    
    # Sort by absolute correlation strength
    significant_pairs.sort(key=lambda x: abs(x[3]), reverse=True)
    
    # Plot significant pairs
    for dim_qx, dim_qy, filt_data, corr_ken, p_ken_raw, p_ken_fdr, n_points in significant_pairs[:len(axes)]:
        ax = axes[plot_idx]
        
        x_data = filt_data[f'{dim_qx}_1']
        y_data = filt_data[f'{dim_qy}_2']
        
        # Create scatter plot
        ax.scatter(x_data, y_data, alpha=0.6, s=30, color='steelblue')
        
        # Add trend line for FDR-significant correlations
        if not np.isnan(p_ken_fdr) and p_ken_fdr < 0.05:
            z = np.polyfit(x_data, y_data, 1)
            p = np.poly1d(z)
            x_trend = np.linspace(x_data.min(), x_data.max(), 100)
            ax.plot(x_trend, p(x_trend), "r-", alpha=0.8, linewidth=2)
        
        # Add statistics with FDR information
        fdr_significance = "*" if (not np.isnan(p_ken_fdr) and p_ken_fdr < 0.05) else ""
        raw_significance = "*" if p_ken_raw < 0.05 else ""
        
        title_text = f'{dim_qx} vs {dim_qy}\ntau={corr_ken:.3f}{fdr_significance}'
        subtitle_text = f'p_raw={p_ken_raw:.3f}{raw_significance}'
        if not np.isnan(p_ken_fdr):
            subtitle_text += f', p_fdr={p_ken_fdr:.3f}{fdr_significance}'
        subtitle_text += f', n={n_points}'
        
        ax.set_title(title_text, fontsize=10, fontweight='bold')
        ax.text(0.5, -0.15, subtitle_text, transform=ax.transAxes, 
                ha='center', fontsize=8, style='italic')
        
        ax.set_xlabel(dim_qx, fontsize=9)
        ax.set_ylabel(dim_qy, fontsize=9)
        ax.grid(True, alpha=0.3)
        
        plot_idx += 1
    
    # Hide unused subplots
    for idx in range(plot_idx, len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    plt.savefig(os.path.join(mainfolder, "filtered_significant_dataclouds_fdr_fifteen_min_timestitch.png"), bbox_inches='tight', dpi=300)
    plt.show()

# Updated function calls with FDR correction:

print("Creating full datacloud matrix with FDR correction...")
create_datacloud_matrix(subOb_dict, list(subOb_dict.keys()), mainfolder, 
                        kendall_mat_all_day, kendall_pval_mat_all_day, 
                        kendall_pval_mat_all_day_fdr, kendall_len_mat_all_day)

#print("Creating condensed datacloud matrix with FDR correction...")
#create_condensed_datacloud_matrix(subOb_dict, list(subOb_dict.keys()), mainfolder,
                                 #kendall_mat_all_day, kendall_pval_mat_all_day,
                                 #kendall_pval_mat_all_day_fdr, kendall_len_mat_all_day)
                            

In [ ]:
"""
SAME AS ABOVE BUT WITH LAG 
"""

lag = 32

focus_folder = r'dataclouds_CAM_LMU\Stream_ASD_001\all_days'
folder_req = os.path.join(focus_folder, 'lag', str(lag))


if not os.path.exists(folder_req):
    os.makedirs(folder_req, exist_ok=True)
focused_folder = folder_req


def giv_date_and_day(reqYear, reqMonth, reqDay):
    date_obj = datetime(reqYear, reqMonth, reqDay)
    return date_obj.strftime("%Y-%m-%d\n%a")  

def lagged_kendall_tau(df, lag):
    # Shift one column by lag (forward lag; negative for backward lag)
    shifted = df.copy()
    col1 = df.columns[0]
    col2 = df.columns[1]
    shifted[col1] = shifted[col1].shift(lag)
    # Drop rows with any NaN (either initial or from shift)
    clean = shifted.dropna(subset=[col1, col2])
    nData = len(clean)
    
    stat1, pval1, stat2, pval2, tau, p_value = np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
    if len(clean)>=3:
        stat1, pval1 = shapiro(clean[col1])
        stat2, pval2 = shapiro(clean[col2])
        # Compute Kendall's tau
        tau, p_value = kendalltau(clean[col1], clean[col2])
    return stat1, pval1, stat2, pval2, tau, p_value, nData

def lagged_per_day_vals(df, lag):
    # Shift one column by lag (forward lag; negative for backward lag)
    shifted = df.copy()
    shifted.replace(-5000, np.nan, inplace=True)
    col1 = df.columns[0]
    col2 = df.columns[1]
    shifted[col1] = shifted[col1].shift(lag)
    # Drop rows with any NaN (either initial or from shift)
    clean = shifted.dropna(subset=[col1, col2])
    return clean


shapiro_mat_all_day = pd.DataFrame(np.zeros((len(subOb), len(subOb))), columns=subOb, index=subOb)
shapiro_pval_mat_all_day = pd.DataFrame(np.zeros((len(subOb), len(subOb))), columns=subOb, index=subOb)
#for correlation test
kendall_mat_all_day = pd.DataFrame(np.zeros((len(subOb), len(subOb))), columns=subOb, index=subOb)
kendall_pval_mat_all_day = pd.DataFrame(np.zeros((len(subOb), len(subOb))), columns=subOb, index=subOb)
kendall_len_mat_all_day = pd.DataFrame(np.zeros((len(subOb), len(subOb))), columns=subOb, index=subOb)
#for fdr correction
kendall_pval_mat_all_day_fdr = pd.DataFrame(np.zeros((len(subOb), len(subOb))), columns=subOb, index=subOb)
  
for dim_qx in subOb_dict:
    for dim_qy in subOb_dict:
        #print(dim_qx , dim_qy)
        dimqx = []
        dimqy = []
        for subfolder in subOb_dict[dim_qx].keys():
            if subfolder in subOb_dict[dim_qy].keys():
                #print(subfolder)
                #print(f'Appending data for common subfolder: {subfolder} for dims {dim_qx} and {dim_qy}')
                #creating a temporary store of per day values. The temp_dimqx will be shifted by the required lag and then both lists will be appended to dimqx and dimqy
                temp_dimqx = []
                temp_dimqy = []
                for val_x in subOb_dict[dim_qx][subfolder].values():
                    temp_dimqx.append(val_x)
                for val_y in subOb_dict[dim_qy][subfolder].values():
                    temp_dimqy.append(val_y)
                temp_df = pd.DataFrame({f'{dim_qx}_1': temp_dimqx, f'{dim_qy}_2': temp_dimqy})
                clean_temp_df = lagged_per_day_vals(temp_df, lag)
                for val_x in clean_temp_df[f'{dim_qx}_1']:
                    #print('appending shifted values of this day of the dimension: ', f'{dim_qx}')
                    dimqx.append(val_x)
                for val_y in clean_temp_df[f'{dim_qy}_2']:
                    #print('appending non-shifted values of this day of the dimension: ', f'{dim_qy}')
                    dimqy.append(val_y)
                

        filt_data = pd.DataFrame({f'{dim_qx}_1' : dimqx, f'{dim_qy}_2' : dimqy}) 
        
        
        if len(filt_data)>=2:
                stat1, pval1 = shapiro(filt_data[f'{dim_qx}_1'])
                stat2, pval2 = shapiro(filt_data[f'{dim_qy}_2'])
                corr_ken, p_ken = kendalltau(filt_data[f'{dim_qx}_1'], filt_data[f'{dim_qy}_2'])
                if pval1 > 0.05 and pval2 > 0.05:
                    shapiro_pval_mat_all_day.loc[dim_qx, dim_qy] = 1
                else:
                    shapiro_pval_mat_all_day.loc[dim_qx, dim_qy] = 0
                kendall_mat_all_day.loc[dim_qx, dim_qy] = corr_ken
                kendall_pval_mat_all_day.loc[dim_qx, dim_qy] = p_ken
                kendall_len_mat_all_day.loc[dim_qx, dim_qy] = len(filt_data)
        else:
                #not enough data to test normality or calculate kendall corr
                shapiro_pval_mat_all_day.loc[dim_qx, dim_qy] = np.nan
                kendall_mat_all_day.loc[dim_qx, dim_qy] = np.nan
                kendall_pval_mat_all_day.loc[dim_qx, dim_qy] = np.nan
                kendall_len_mat_all_day.loc[dim_qx, dim_qy] = len(filt_data)

#Applying fdr correction
#
temp_store = []
pPreFdr = []
pos = []
for meas_1 in subOb_dict:
    temp_store.append(meas_1)
    for meas_2 in subOb_dict:
        #print(meas_1, meas_2)
        
        if not np.isnan(kendall_mat_all_day.loc[meas_1, meas_2]):
                pPreFdr.append(kendall_pval_mat_all_day.loc[meas_1, meas_2])
                pos.append((meas_1, meas_2))

pPostFdr = multipletests(pPreFdr, method='fdr_bh')[1]

for (meas_1, meas_2), p_adj in zip(pos, pPostFdr):
    kendall_pval_mat_all_day_fdr.loc[meas_1, meas_2] = p_adj
    kendall_pval_mat_all_day_fdr.loc[meas_2, meas_1] = p_adj


#creating a custom annotation matrix combining normality test, tau, p-value, and data length
annotations = kendall_mat_all_day.copy()
for dim_qx in subOb_dict:
    for dim_qy in subOb_dict:
        if not np.isnan(kendall_mat_all_day.loc[dim_qx, dim_qy]):
            annotations.loc[dim_qx, dim_qy] = f"tau={kendall_mat_all_day.loc[dim_qx, dim_qy]:.2f}\n"
            if kendall_pval_mat_all_day.loc[dim_qx, dim_qy] < 0.05:
                annotations.loc[dim_qx, dim_qy] += f"p={kendall_pval_mat_all_day.loc[dim_qx, dim_qy]:.3f}*\n"
            else:
                annotations.loc[dim_qx, dim_qy] += f"p={kendall_pval_mat_all_day.loc[dim_qx, dim_qy]:.3f}\n"
            annotations.loc[dim_qx, dim_qy] += f"dl={int(kendall_len_mat_all_day.loc[dim_qx, dim_qy])}\n"
            annotations.loc[dim_qx, dim_qy] += f"norm={int(shapiro_pval_mat_all_day.loc[dim_qx, dim_qy])}\n"
            annotations.loc[dim_qx, dim_qy] += f"lag_dim={dim_qx}"
        else:
            annotations.loc[dim_qx, dim_qy] = ""

# Plot the combined heatmap with custom annotations (tau, p-value, and data length)
plt.figure(figsize=(32, 32))
sns.heatmap(kendall_mat_all_day, annot=annotations, fmt="", cmap='coolwarm', square=True, cbar=True, mask=np.isnan(kendall_mat_all_day), annot_kws={"size": 12})
title = 'Kendall Correlation, p-value, and Data Length for participant: '+ mainfolder.split("\\")[-1]
plt.title(title)
plt.savefig(os.path.join(focused_folder, "focused_all_day_subjective_objective_correlogram_fifteen_min_timestitch.png"), bbox_inches='tight', dpi=300)
plt.show()

#repeating annotations and figure for fdr correction
annotations_fdr = kendall_mat_all_day.copy()
for meas_1 in subOb_dict:
    for meas_2 in subOb_dict:
        if not np.isnan(kendall_mat_all_day.loc[meas_1, meas_2]):
            annotations_fdr.loc[meas_1, meas_2] = f"tau={kendall_mat_all_day.loc[meas_1, meas_2]:.2f}\n"
            if kendall_pval_mat_all_day_fdr.loc[meas_1, meas_2] < 0.05:
                annotations_fdr.loc[meas_1, meas_2] += f"p={kendall_pval_mat_all_day_fdr.loc[meas_1, meas_2]:.2f}*\n"
            else:
                annotations_fdr.loc[meas_1, meas_2] += f"p={kendall_pval_mat_all_day_fdr.loc[meas_1, meas_2]:.2f}\n"
            annotations_fdr.loc[meas_1, meas_2] += f"dl={int(kendall_len_mat_all_day.loc[meas_1, meas_2])}\n"
            annotations_fdr.loc[meas_1, meas_2] += f"norm={int(shapiro_pval_mat_all_day.loc[meas_1, meas_2])}\n"
            annotations_fdr.loc[meas_1, meas_2] += f"lag_dim={meas_1}"
        else:
            annotations_fdr.loc[meas_1, meas_2] = ""

# Plot the combined heatmap with custom annotations (tau, p-value, and data length)
plt.figure(figsize=(32, 32))
sns.heatmap(kendall_mat_all_day, annot=annotations_fdr, fmt="", cmap='coolwarm', square=True, cbar=True, mask=np.isnan(kendall_mat_all_day), annot_kws={"size": 12})
title = 'Kendall Correlation, p-value (fdr corrected), and Data Length for participant: '+ mainfolder.split("\\")[-1]
plt.title(title)
plt.savefig(os.path.join(focused_folder, "focused_all_day_subjective_objective_correlogram_fifteen_min_fdr_timestitch.png"), bbox_inches='tight', dpi=300)
plt.show()

#trying out dataclouds with FDR correction

def create_datacloud_matrix(subOb_dict, subOb, mainfolder, kendall_mat_all_day, kendall_pval_mat_all_day, kendall_pval_mat_all_day_fdr, kendall_len_mat_all_day,lag, focused_folder, min_points=10):
    """
    Create a matrix of scatter plots (dataclouds) corresponding to correlation analysis
    Now uses FDR-corrected p-values for significance
    """
    def lagged_per_day_vals(df, lag):
        # Shift one column by lag (forward lag; negative for backward lag)
        shifted = df.copy()
        shifted.replace(-5000, np.nan, inplace=True)
        col1 = df.columns[0]
        col2 = df.columns[1]
        shifted[col1] = shifted[col1].shift(lag)
        # Drop rows with any NaN (either initial or from shift)
        clean = shifted.dropna(subset=[col1, col2])
        return clean

    
    special_var = {'eda': 30, 'pulse_rate':150, 'pulse_rate_variability':300, 'resp_rate':30, 'acc_std_dev':0.5}  #change AS REQUIRED 
    n_vars = len(subOb)
    
    # Create figure with subplots
    fig, axes = plt.subplots(n_vars, n_vars, figsize=(3*n_vars, 2*n_vars))
    participant_title_list = mainfolder.split("\\")[-1].split('_')[0:3]
    participant_title = '_'.join(participant_title_list)
    suptitle = 'Datacloud Matrix (FDR-corrected) for participant: '+ participant_title
    fig.suptitle(suptitle, fontsize=26, y=0.98)
    
    # If only one variable, make axes 2D for consistent indexing
    if n_vars == 1:
        axes = np.array([[axes]])
    elif n_vars == 2:
        axes = axes.reshape(2, 2)
    
    for i, dim_qx in enumerate(subOb):
        for j, dim_qy in enumerate(subOb):
            ax = axes[i, j]
            
            # Collect data for this pair
            dimqx = []
            dimqy = []
            
            for subfolder in subOb_dict[dim_qx].keys():
                if subfolder in subOb_dict[dim_qy].keys():
                    temp_dimqx = []
                    temp_dimqy = []
                    for val_x in subOb_dict[dim_qx][subfolder].values():
                        temp_dimqx.append(val_x)
                    for val_y in subOb_dict[dim_qy][subfolder].values():
                        temp_dimqy.append(val_y)
                    temp_df = pd.DataFrame({f'{dim_qx}_1': temp_dimqx, f'{dim_qy}_2': temp_dimqy})
                    clean_temp_df = lagged_per_day_vals(temp_df, lag)
                    for val_x in clean_temp_df[f'{dim_qx}_1']:
                        #print('appending shifted values of this day of the dimension: ', f'{dim_qx}')
                        dimqx.append(val_x)
                    for val_y in clean_temp_df[f'{dim_qy}_2']:
                        #print('appending non-shifted values of this day of the dimension: ', f'{dim_qy}')
                        dimqy.append(val_y)
            
            # Create DataFrame and filter data
            filt_data = pd.DataFrame({f'{dim_qx}_1' : dimqx, f'{dim_qy}_2' : dimqy})
            
            
            if len(filt_data) >= 2:
                x_data = filt_data[f'{dim_qx}_1']
                y_data = filt_data[f'{dim_qy}_2']
                # Diagonal - do not leave blank this time for lagged correlations
                # Create scatter plot
                ax.scatter(x_data, y_data, alpha=0.6, s=20, color='steelblue')
                if dim_qx in special_var.keys():
                        for key, val in special_var.items():
                            if dim_qx == key:
                                ax.set_xlim(0, val)
                else:
                    ax.set_xlim(0, 4)
                if dim_qy in special_var.keys():
                        for key, val in special_var.items():
                            if dim_qy == key:
                                ax.set_ylim(0, val)
                else:
                    ax.set_ylim(0, 4)    

                # Get correlation statistics from existing matrices
                if len(filt_data) >= min_points and not np.isnan(kendall_mat_all_day.loc[dim_qx, dim_qy]):
                        corr_ken = kendall_mat_all_day.loc[dim_qx, dim_qy]
                        p_ken_raw = kendall_pval_mat_all_day.loc[dim_qx, dim_qy]
                        p_ken_fdr = kendall_pval_mat_all_day_fdr.loc[dim_qx, dim_qy]
                        n_points = int(kendall_len_mat_all_day.loc[dim_qx, dim_qy])
                    
                        
                        # Add trend line if FDR-corrected correlation is significant and reasonably strong
                        if not np.isnan(p_ken_fdr) and p_ken_fdr < 0.05 : #and abs(corr_ken) > 0.2
                            # Fit linear trend line
                            z = np.polyfit(x_data, y_data, 1)
                            p = np.poly1d(z)
                            ax.plot(x_data.sort_values(), p(x_data.sort_values()), 
                                   "r--", alpha=0.8, linewidth=1.5)
                        
                        # Add correlation info with both raw and FDR p-values
                        fdr_sig = "*" if (not np.isnan(p_ken_fdr) and p_ken_fdr < 0.05) else ""
                        raw_sig = "*" if p_ken_raw < 0.05 else ""
                        
                        annotation_text = f'tau={corr_ken:.3f}{fdr_sig}\n'
                        annotation_text += f'p_raw={p_ken_raw:.3f}{raw_sig}\n'
                        if not np.isnan(p_ken_fdr):
                            annotation_text += f'p_fdr={p_ken_fdr:.3f}{fdr_sig}\n'
                        else:
                            annotation_text += f'p_fdr=n/a\n'
                        annotation_text += f'n={n_points}'
                        
                        ax.text(0.05, 0.95, annotation_text,
                               transform=ax.transAxes, fontsize=7,
                               verticalalignment='top',
                               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
                    
                ax.set_xlabel(dim_qx+'_lag_'+str(lag), fontsize=9)
                ax.set_ylabel(dim_qy, fontsize=9)
                    
                
            else:
                # Not enough data
                ax.text(0.5, 0.5, f'Insufficient data\n(n={len(filt_data)})', 
                       transform=ax.transAxes, ha='center', va='center',
                       fontsize=10, color='red')
                ax.set_xlabel(dim_qx, fontsize=9)
                ax.set_ylabel(dim_qy, fontsize=9)
                if dim_qx == special_var:
                    ax.set_xlim(0, 30)
                else:
                    ax.set_xlim(0, 4)
                if dim_qy == special_var:
                    ax.set_ylim(0, 30)
                else:
                    ax.set_ylim(0, 4)
            
            # Clean up axes
            ax.tick_params(labelsize=8)
            ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(focused_folder, "focused_datacloud_matrix_fdr_fifteen_min_timestitch.png"), bbox_inches='tight', dpi=300)
    plt.show()


def create_condensed_datacloud_matrix(subOb_dict, subOb, mainfolder, kendall_mat_all_day, kendall_pval_mat_all_day, kendall_pval_mat_all_day_fdr, kendall_len_mat_all_day, min_points=10):
    """
    OPTIONALLY ADD SPECIFIC LIMITS LIKE IN THE PREVIOUS FUNCTION IF USING THIS FUNCTION
    Create a condensed matrix showing only unique pairs (lower triangle)
    Now uses FDR-corrected p-values for significance
    """
    # Calculate number of unique pairs
    n_vars = len(subOb)
    n_unique_pairs = n_vars * (n_vars - 1) // 2
    
    if n_unique_pairs == 0:
        print("Need at least 2 variables for correlation dataclouds")
        return
    
    # Determine subplot layout
    cols = min(5, n_unique_pairs)  # Max 5 columns
    rows = (n_unique_pairs + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 4*rows))
    suptitle = 'Significant Correlations (FDR-corrected) - Dataclouds for participant: '+mainfolder.split("\\")[-1]
    fig.suptitle(suptitle, fontsize=14)
    
    # Make axes iterable
    if n_unique_pairs == 1:
        axes = [axes]
    else:
        axes = axes.flatten() if hasattr(axes, 'flatten') else axes
    
    plot_idx = 0
    significant_pairs = []
    
    # Collect all pairs first (lower triangle)
    for i, dim_qx in enumerate(subOb):
        for j, dim_qy in enumerate(subOb):
            if i <= j:  # Only lower triangle
                continue
                
            # Collect and filter data
            dimqx = []
            dimqy = []
            
            for subfolder in subOb_dict[dim_qx].keys():
                if subfolder in subOb_dict[dim_qy].keys():
                    for val_x in subOb_dict[dim_qx][subfolder].values():
                        dimqx.append(val_x)
                    for val_y in subOb_dict[dim_qy][subfolder].values():
                        dimqy.append(val_y)
            
            df = pd.DataFrame({f'{dim_qx}_1': dimqx, f'{dim_qy}_2': dimqy})
            df.replace(-5000, np.nan, inplace=True)
            filt_data = df.dropna()
            
            if len(filt_data) >= min_points and not np.isnan(kendall_mat_all_day.loc[dim_qx, dim_qy]):
                corr_ken = kendall_mat_all_day.loc[dim_qx, dim_qy]
                p_ken_raw = kendall_pval_mat_all_day.loc[dim_qx, dim_qy]
                p_ken_fdr = kendall_pval_mat_all_day_fdr.loc[dim_qx, dim_qy]
                n_points = int(kendall_len_mat_all_day.loc[dim_qx, dim_qy])
                significant_pairs.append((dim_qx, dim_qy, filt_data, corr_ken, p_ken_raw, p_ken_fdr, n_points))
    
    # Sort by absolute correlation strength
    significant_pairs.sort(key=lambda x: abs(x[3]), reverse=True)
    
    # Plot significant pairs
    for dim_qx, dim_qy, filt_data, corr_ken, p_ken_raw, p_ken_fdr, n_points in significant_pairs[:len(axes)]:
        ax = axes[plot_idx]
        
        x_data = filt_data[f'{dim_qx}_1']
        y_data = filt_data[f'{dim_qy}_2']
        
        # Create scatter plot
        ax.scatter(x_data, y_data, alpha=0.6, s=30, color='steelblue')
        
        # Add trend line for FDR-significant correlations
        if not np.isnan(p_ken_fdr) and p_ken_fdr < 0.05:
            z = np.polyfit(x_data, y_data, 1)
            p = np.poly1d(z)
            x_trend = np.linspace(x_data.min(), x_data.max(), 100)
            ax.plot(x_trend, p(x_trend), "r-", alpha=0.8, linewidth=2)
        
        # Add statistics with FDR information
        fdr_significance = "*" if (not np.isnan(p_ken_fdr) and p_ken_fdr < 0.05) else ""
        raw_significance = "*" if p_ken_raw < 0.05 else ""
        
        title_text = f'{dim_qx} vs {dim_qy}\ntau={corr_ken:.3f}{fdr_significance}'
        subtitle_text = f'p_raw={p_ken_raw:.3f}{raw_significance}'
        if not np.isnan(p_ken_fdr):
            subtitle_text += f', p_fdr={p_ken_fdr:.3f}{fdr_significance}'
        subtitle_text += f', n={n_points}'
        
        ax.set_title(title_text, fontsize=10, fontweight='bold')
        ax.text(0.5, -0.15, subtitle_text, transform=ax.transAxes, 
                ha='center', fontsize=8, style='italic')
        
        ax.set_xlabel(dim_qx, fontsize=9)
        ax.set_ylabel(dim_qy, fontsize=9)
        ax.grid(True, alpha=0.3)
        
        plot_idx += 1
    
    # Hide unused subplots
    for idx in range(plot_idx, len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    plt.savefig(os.path.join(mainfolder, "filtered_significant_dataclouds_fdr_fifteen_min_timestitch.png"), bbox_inches='tight', dpi=300)
    plt.show()

# Updated function calls with FDR correction:

print("Creating full datacloud matrix with FDR correction...")
create_datacloud_matrix(subOb_dict, list(subOb_dict.keys()), mainfolder, 
                        kendall_mat_all_day, kendall_pval_mat_all_day, 
                        kendall_pval_mat_all_day_fdr, kendall_len_mat_all_day, lag, focused_folder)

#print("Creating condensed datacloud matrix with FDR correction...")
#create_condensed_datacloud_matrix(subOb_dict, list(subOb_dict.keys()), mainfolder,
                                 #kendall_mat_all_day, kendall_pval_mat_all_day,
                                 #kendall_pval_mat_all_day_fdr, kendall_len_mat_all_day)
                            